## Open and View Datasets

In [762]:
# Modify if different

path_activity = "activity_data.csv"
path_customer = "customer_data.csv"
path_complaints = "complaints.xlsx"

In [763]:
import pandas as pd

# Loading files
activity_data = pd.read_csv(path_activity)
customer_data = pd.read_csv(path_customer)
complaints_data = pd.read_excel(path_complaints)

In [764]:
# Unify column name convention

activity_data.columns = activity_data.columns.str.lower().str.replace(' ', '_', regex=False)
customer_data.columns = customer_data.columns.str.lower().str.replace(' ', '_', regex=False)
complaints_data.columns = complaints_data.columns.str.lower().str.replace(' ', '_', regex=False)

In [765]:
# Displaying the first 5 rows of each DataFrame
print("Activity Data:")
print(activity_data.head())

print("Customer Data:")
print(customer_data.head())

print("Complaints:")
print(complaints_data.head())

Activity Data:
   customer_id      month  data_usage  phone_usage  use_app
0        10000  1/01/2021       43.61      4570.12        1
1        10001  1/01/2021        2.07      2038.61        0
2        10002  1/01/2021       45.69      1786.97        1
3        10003  1/01/2021       45.70      2450.95        1
4        10004  1/01/2021       15.28      4627.57        1
Customer Data:
   customer_id  birth_date   join_date      plan_type  churn_in_3mos
0        10000  1994-08-13  2015-11-22       postpaid              0
1        10001  1994-06-25  2015-01-12  pay-as-you-go              1
2        10002  2008-06-10  2020-05-22        prepaid              0
3        10003  1970-09-04  2017-11-10        prepaid              0
4        10004  1969-11-06  2019-05-19        prepaid              0
Complaints:
   customer_id                                                                                                                                                                          

## Initial Analysis and Data Cleaning

There are some issues with the data, on first glass we suggest solving them by:
- Check for Nulls
- Look into datatypes and handle datetime data types
- Merging the data with the index customer_ID requires handling the month variable in order to not replicate rows: OHE
- One Hot Encode the variable plan_type
- Create new variables for average data_usage and average_phone usage
- Create new variables for age at joint_date and Years_On_Plan (01/2022 - Joint_date)

### Assessing data types (transforming variables into datetime)

In [766]:
# Initial data type check
print("Activity Data Types:")
print(activity_data.dtypes)

print("Customer Data Types:")
print(customer_data.dtypes)

print("Complaints Data Types:")
print(complaints_data.dtypes)

Activity Data Types:
customer_id      int64
month           object
data_usage     float64
phone_usage    float64
use_app          int64
dtype: object
Customer Data Types:
customer_id       int64
birth_date       object
join_date        object
plan_type        object
churn_in_3mos     int64
dtype: object
Complaints Data Types:
customer_id     int64
complaint      object
category       object
dtype: object


In [767]:
# Convert date columns to datetime format
activity_data['month'] = pd.to_datetime(activity_data['month'], dayfirst=True)
customer_data['birth_date'] = pd.to_datetime(customer_data['birth_date'])
customer_data['join_date'] = pd.to_datetime(customer_data['join_date'])

The first step of the data cleaning is composed of converting the dates into the correct data type: datetime

In [768]:
# Initial check if number of rows and customer_IDs are identical
activity_in_customer = set(activity_data["customer_id"]).issubset(set(customer_data["customer_id"]))
print("All customer IDs in activity_data are in customer_data:", activity_in_customer)

All customer IDs in activity_data are in customer_data: True


### Checking for nulls

In [769]:
# Checking for nulls in Activity Data
print("Missing values in Activity Data:")
print(activity_data.isnull().sum())

# Checking for nulls in Customer Data
print("Missing values in Customer Data:")
print(customer_data.isnull().sum())

# Checking for nulls in Customer Data
print("Missing values in Customer Data:")
print(complaints_data.isnull().sum())

Missing values in Activity Data:
customer_id    0
month          0
data_usage     0
phone_usage    0
use_app        0
dtype: int64
Missing values in Customer Data:
customer_id      0
birth_date       0
join_date        0
plan_type        0
churn_in_3mos    0
dtype: int64
Missing values in Customer Data:
customer_id    0
complaint      0
category       0
dtype: int64


No nulls, we can proceed, in case there are nulls they must be addressed. 

### Data Frame Activity_Data

In [770]:
print(activity_data.head())

   customer_id      month  data_usage  phone_usage  use_app
0        10000 2021-01-01       43.61      4570.12        1
1        10001 2021-01-01        2.07      2038.61        0
2        10002 2021-01-01       45.69      1786.97        1
3        10003 2021-01-01       45.70      2450.95        1
4        10004 2021-01-01       15.28      4627.57        1


In [771]:
# Checking the number of rows, after an initial analysis it appears that each customer has a seperate line per month
num_rows = activity_data.shape[0]
print(f"Number of rows in activity_data: {num_rows}")

# This is indeed the case

Number of rows in activity_data: 120000


#### OHE the column Month to prevent duplicated rows when merging

In [772]:
# Create a pivot table
activity_pivot = activity_data.pivot_table(
    index='customer_id',
    columns='month',
    values=['data_usage', 'phone_usage', 'use_app'],
    aggfunc='first'
)

# Flatten the MultiIndex in columns and format them as 'metric_monthname'
activity_pivot.columns = ['{}_{}'.format(metric, month) for metric, month in activity_pivot.columns]

# Define the correct order of months
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

# Reorder columns based on month order
month_columns = sorted([col for col in activity_pivot.columns if any(m in col for m in month_order)], 
                       key=lambda x: month_order.index(x.split('_')[-1]))

# Keep customer_id at the start
other_columns = [col for col in activity_pivot.columns if col not in month_columns]

# Reassign columns in the correct order
activity_pivot = activity_pivot[other_columns + month_columns]

# Reset index to make 'customer_id' a column again, if it is set as index
activity_pivot.reset_index(inplace=True)

# Display the first few rows
print(activity_pivot.head())


   customer_id  data_usage_2021-01-01 00:00:00  data_usage_2021-02-01 00:00:00  data_usage_2021-03-01 00:00:00  data_usage_2021-04-01 00:00:00  data_usage_2021-05-01 00:00:00  data_usage_2021-06-01 00:00:00  data_usage_2021-07-01 00:00:00  data_usage_2021-08-01 00:00:00  data_usage_2021-09-01 00:00:00  data_usage_2021-10-01 00:00:00  data_usage_2021-11-01 00:00:00  data_usage_2021-12-01 00:00:00  phone_usage_2021-01-01 00:00:00  phone_usage_2021-02-01 00:00:00  phone_usage_2021-03-01 00:00:00  phone_usage_2021-04-01 00:00:00  phone_usage_2021-05-01 00:00:00  phone_usage_2021-06-01 00:00:00  phone_usage_2021-07-01 00:00:00  phone_usage_2021-08-01 00:00:00  phone_usage_2021-09-01 00:00:00  phone_usage_2021-10-01 00:00:00  phone_usage_2021-11-01 00:00:00  phone_usage_2021-12-01 00:00:00  use_app_2021-01-01 00:00:00  use_app_2021-02-01 00:00:00  use_app_2021-03-01 00:00:00  use_app_2021-04-01 00:00:00  use_app_2021-05-01 00:00:00  use_app_2021-06-01 00:00:00  use_app_2021-07-01 00:00:00  u

In [773]:
# Count the number of rows
activity_data = activity_pivot
num_rows = activity_data.shape[0]
print(f"Number of rows in activity_pivot: {num_rows}")

Number of rows in activity_pivot: 10000


In [774]:
activity_data

,customer_id,data_usage_2021-01-01 00:00:00,data_usage_2021-02-01 00:00:00,data_usage_2021-03-01 00:00:00,data_usage_2021-04-01 00:00:00,data_usage_2021-05-01 00:00:00,data_usage_2021-06-01 00:00:00,data_usage_2021-07-01 00:00:00,data_usage_2021-08-01 00:00:00,data_usage_2021-09-01 00:00:00,data_usage_2021-10-01 00:00:00,data_usage_2021-11-01 00:00:00,data_usage_2021-12-01 00:00:00,phone_usage_2021-01-01 00:00:00,phone_usage_2021-02-01 00:00:00,phone_usage_2021-03-01 00:00:00,phone_usage_2021-04-01 00:00:00,phone_usage_2021-05-01 00:00:00,phone_usage_2021-06-01 00:00:00,phone_usage_2021-07-01 00:00:00,phone_usage_2021-08-01 00:00:00,phone_usage_2021-09-01 00:00:00,phone_usage_2021-10-01 00:00:00,phone_usage_2021-11-01 00:00:00,phone_usage_2021-12-01 00:00:00,use_app_2021-01-01 00:00:00,use_app_2021-02-01 00:00:00,use_app_2021-03-01 00:00:00,use_app_2021-04-01 00:00:00,use_app_2021-05-01 00:00:00,use_app_2021-06-01 00:00:00,use_app_2021-07-01 00:00:00,use_app_2021-08-01 00:00:00,use_app_2021-09-01 00:00:00,use_app_2021-10-01 00:00:00,use_app_2021-11-01 00:00:00,use_app_2021-12-01 00:00:00
0,10000,43.61,115.97,84.20,80.87,66.45,34.89,67.70,107.80,105.42,22.15,111.59,82.34,4570.12,4589.49,835.60,4398.57,1060.44,703.84,596.48,4590.48,1934.41,4706.93,4217.64,4010.24,1,1,0,1,0,0,1,1,1,1,1,0
1,10001,2.07,13.87,12.11,3.40,12.22,21.06,4.29,10.17,21.43,8.85,16.34,21.63,2038.61,1952.95,1762.70,1383.54,217.01,174.23,1983.84,799.41,3371.37,2168.73,227.18,4920.68,0,1,1,1,1,1,1,1,1,1,1,1
2,10002,45.69,40.34,33.78,14.24,38.86,57.87,46.47,56.30,7.10,11.48,38.80,49.67,1786.97,4118.81,3137.10,1736.49,3278.96,1962.14,2801.82,166.68,4878.91,4089.35,1397.95,4926.51,1,0,1,0,0,1,0,0,1,1,1,1
3,10003,45.70,27.05,22.06,56.30,44.92,3.72,40.69,39.77,9.22,11.05,35.79,44.51,2450.95,1551.45,2738.69,3627.57,4662.40,1018.62,1742.39,2876.11,1795.95,720.50,4444.18,3381.92,1,1,1,1,1,0,1,1,1,1,1,1
4,10004,15.28,40.19,27.20,17.89,48.10,41.49,43.77,42.63,42.63,46.94,26.22,39.68,4627.57,1785.52,4617.92,2639.51,2159.29,3285.00,2919.12,4636.01,186.52,243.33,2102.91,236.99,1,1,0,1,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19995,14.00,49.02,12.45,16.86,40.69,23.24,48.56,52.59,43.13,35.22,3.47,37.83,176.33,2347.89,794.46,285.20,1462.39,597.70,4891.27,3167.08,3581.08,3454.54,4503.46,1733.67,1,0,0,0,1,1,1,1,1,0,1,1
9996,19996,2.66,56.19,16.14,57.52,29.36,2.60,39.34,54.25,47.30,54.16,44.85,33.76,2636.17,2960.40,2076.95,2106.95,3721.75,2504.46,3901.31,3336.71,195.20,1676.06,3344.52,4220.29,1,1,1,1,1,1,1,1,0,1,1,1
9997,19997,86.61,67.07,34.62,115.72,103.32,75.41,105.82,119.86,73.28,51.29,98.17,11.96,509.70,912.83,4649.21,370.82,4262.41,2247.21,2843.77,4929.77,2948.38,793.92,3831.65,1659.82,0,0,1,0,1,0,1,1,1,1,1,1
9998,19998,35.92,52.72,49.01,36.30,104.63,20.36,91.72,16.12,94.87,22.35,69.39,39.31,3340.94,2361.35,4090.45,3010.06,1699.38,2924.75,4946.29,936.60,4130.59,4559.14,1423.28,4154.94,1,1,1,1,1,0,1,1,1,0,1,0


### Data Frame Client_Data

First we need to check how many unique values are there on the column plan_type

In [775]:
# Count unique values in plan_type column
unique_values = customer_data['plan_type'].value_counts()

# Display results
print("Unique values in 'plan_type' and their counts:")
print(unique_values)

Unique values in 'plan_type' and their counts:
plan_type
prepaid          7040
postpaid         1973
pay-as-you-go     987
Name: count, dtype: int64


In [776]:
# Preserve the original 'plan_type' column before applying get_dummies
customer_data["plan_type_original"] = customer_data["plan_type"]

# Apply One-Hot Encoding without dropping the original 'plan_type' column
customer_data = pd.get_dummies(customer_data, columns=['plan_type'], prefix='plan', dtype=int)

# Rename the preserved column back to 'plan_type'
customer_data.rename(columns={"plan_type_original": "plan_type"}, inplace=True)

# Display the transformed dataset
print(customer_data.head())  # Show the first few rows

   customer_id birth_date  join_date  churn_in_3mos      plan_type  plan_pay-as-you-go  plan_postpaid  plan_prepaid
0        10000 1994-08-13 2015-11-22              0       postpaid                   0              1             0
1        10001 1994-06-25 2015-01-12              1  pay-as-you-go                   1              0             0
2        10002 2008-06-10 2020-05-22              0        prepaid                   0              0             1
3        10003 1970-09-04 2017-11-10              0        prepaid                   0              0             1
4        10004 1969-11-06 2019-05-19              0        prepaid                   0              0             1


#### Creating new variables for the Client Data Frame

Variable: Age when client joined the plan

In [777]:
# Calculate age when joined
customer_data['age_at_joining'] = (customer_data['join_date'] - customer_data['birth_date']).dt.days // 365

# Display the updated dataset
print(customer_data.head())  # Show first few rows

   customer_id birth_date  join_date  churn_in_3mos      plan_type  plan_pay-as-you-go  plan_postpaid  plan_prepaid  age_at_joining
0        10000 1994-08-13 2015-11-22              0       postpaid                   0              1             0              21
1        10001 1994-06-25 2015-01-12              1  pay-as-you-go                   1              0             0              20
2        10002 2008-06-10 2020-05-22              0        prepaid                   0              0             1              11
3        10003 1970-09-04 2017-11-10              0        prepaid                   0              0             1              47
4        10004 1969-11-06 2019-05-19              0        prepaid                   0              0             1              49


Variable: Years on the plan (Present Day - 01/01/2022)

In [778]:
from datetime import datetime

# Define reference date (January 1, 2022)
reference_date = datetime(2022, 1, 1)

# Calculate years on plan
customer_data['years_on_plan'] = (reference_date - customer_data['join_date']).dt.days / 365

# Display the updated dataset
print(customer_data.head())  # Show first few rows

   customer_id birth_date  join_date  churn_in_3mos      plan_type  plan_pay-as-you-go  plan_postpaid  plan_prepaid  age_at_joining  years_on_plan
0        10000 1994-08-13 2015-11-22              0       postpaid                   0              1             0              21       6.115068
1        10001 1994-06-25 2015-01-12              1  pay-as-you-go                   1              0             0              20       6.975342
2        10002 2008-06-10 2020-05-22              0        prepaid                   0              0             1              11       1.613699
3        10003 1970-09-04 2017-11-10              0        prepaid                   0              0             1              47       4.145205
4        10004 1969-11-06 2019-05-19              0        prepaid                   0              0             1              49       2.624658


### Data Frame Complaint Data

In [779]:
complaints_data.shape

(3500, 3)

In [780]:
complaints_data['customer_id'].nunique()

3500

In [781]:
complaints_data['category'].value_counts()

category
Billing Issue           770
Connection Problem      750
Unauthorized Charges    748
Service Disruption      728
Promotional Issue       504
Name: count, dtype: int64

### Merging the Dataframes

Checking if number of rows is the same on both data frames

In [782]:
# Variables for number of rows
activity_rows = activity_data.shape[0]
customer_rows = customer_data.shape[0]

print(f"Number of rows in Activity Data: {activity_rows}")
print(f"Number of rows in Customer Data: {customer_rows}")

# Checking number of rows for both 
if activity_rows == customer_rows:
    print("\n The number of rows is the same in both datasets.")
else:
    print("\n The number of rows is different between the datasets.")

Number of rows in Activity Data: 10000
Number of rows in Customer Data: 10000

 The number of rows is the same in both datasets.


In [783]:
# Checking if the customer_id column is identical, as it is going to be set as index
if set(activity_data['customer_id']) == set(customer_data['customer_id']):
    print("\n The customer_id values are identical in both datasets. ✅")
else:
    print("\n The customer_id values are NOT identical in both datasets.")



 The customer_id values are identical in both datasets. ✅


#### Merging

In [784]:
customer_data.head() 

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan
0,10000,1994-08-13,2015-11-22,0,postpaid,0,1,0,21,6.115068
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342
2,10002,2008-06-10,2020-05-22,0,prepaid,0,0,1,11,1.613699
3,10003,1970-09-04,2017-11-10,0,prepaid,0,0,1,47,4.145205
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658


In [785]:
activity_data.head()

,customer_id,data_usage_2021-01-01 00:00:00,data_usage_2021-02-01 00:00:00,data_usage_2021-03-01 00:00:00,data_usage_2021-04-01 00:00:00,data_usage_2021-05-01 00:00:00,data_usage_2021-06-01 00:00:00,data_usage_2021-07-01 00:00:00,data_usage_2021-08-01 00:00:00,data_usage_2021-09-01 00:00:00,data_usage_2021-10-01 00:00:00,data_usage_2021-11-01 00:00:00,data_usage_2021-12-01 00:00:00,phone_usage_2021-01-01 00:00:00,phone_usage_2021-02-01 00:00:00,phone_usage_2021-03-01 00:00:00,phone_usage_2021-04-01 00:00:00,phone_usage_2021-05-01 00:00:00,phone_usage_2021-06-01 00:00:00,phone_usage_2021-07-01 00:00:00,phone_usage_2021-08-01 00:00:00,phone_usage_2021-09-01 00:00:00,phone_usage_2021-10-01 00:00:00,phone_usage_2021-11-01 00:00:00,phone_usage_2021-12-01 00:00:00,use_app_2021-01-01 00:00:00,use_app_2021-02-01 00:00:00,use_app_2021-03-01 00:00:00,use_app_2021-04-01 00:00:00,use_app_2021-05-01 00:00:00,use_app_2021-06-01 00:00:00,use_app_2021-07-01 00:00:00,use_app_2021-08-01 00:00:00,use_app_2021-09-01 00:00:00,use_app_2021-10-01 00:00:00,use_app_2021-11-01 00:00:00,use_app_2021-12-01 00:00:00
0,10000,43.61,115.97,84.20,80.87,66.45,34.89,67.70,107.80,105.42,22.15,111.59,82.34,4570.12,4589.49,835.60,4398.57,1060.44,703.84,596.48,4590.48,1934.41,4706.93,4217.64,4010.24,1,1,0,1,0,0,1,1,1,1,1,0
1,10001,2.07,13.87,12.11,3.40,12.22,21.06,4.29,10.17,21.43,8.85,16.34,21.63,2038.61,1952.95,1762.70,1383.54,217.01,174.23,1983.84,799.41,3371.37,2168.73,227.18,4920.68,0,1,1,1,1,1,1,1,1,1,1,1
2,10002,45.69,40.34,33.78,14.24,38.86,57.87,46.47,56.30,7.10,11.48,38.80,49.67,1786.97,4118.81,3137.10,1736.49,3278.96,1962.14,2801.82,166.68,4878.91,4089.35,1397.95,4926.51,1,0,1,0,0,1,0,0,1,1,1,1
3,10003,45.70,27.05,22.06,56.30,44.92,3.72,40.69,39.77,9.22,11.05,35.79,44.51,2450.95,1551.45,2738.69,3627.57,4662.40,1018.62,1742.39,2876.11,1795.95,720.50,4444.18,3381.92,1,1,1,1,1,0,1,1,1,1,1,1
4,10004,15.28,40.19,27.20,17.89,48.10,41.49,43.77,42.63,42.63,46.94,26.22,39.68,4627.57,1785.52,4617.92,2639.51,2159.29,3285.00,2919.12,4636.01,186.52,243.33,2102.91,236.99,1,1,0,1,1,1,1,1,0,1,0,1


In [786]:
complaints_data.head()

,customer_id,complaint,category
0,17256,"Subject: Official Complaint.\n\n The undersigned, customer with code 17256, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: My internet connection has been unreliable, frequently dropping at critical moments..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This situation has greatly impacted my ability to carry out work from home.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Connection Problem
1,18490,"Subject: Official Complaint.\n\n The undersigned, customer with code 18490, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: My internet service was down for an entire day, and I received no communication from customer service..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. The lack of resolution is causing frustration and affecting my usage of the service.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Service Disruption
2,14439,"Subject: Official Complaint.\n\n The undersigned, customer with code 14439, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: There are charges on my bill that I don’t recognize, and customer service has not taken any meaningful steps to resolve the issue..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This ongoing problem has caused significant disruptions to my daily activities.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Billing Issue
3,18777,"Subject: Official Complaint.\n\n The undersigned, customer with code 18777, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: I’ve experienced several service interruptions in the past few weeks, making it difficult to rely on your service..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. The lack of resolution is causing frustration and affecting my usage of the service.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Service Disruption
4,16143,"Subject: Official Complaint.\n\n The undersigned, customer with code 16143, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: Despite upgrading to a higher-speed plan, the connection remains slow and unreliable..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This situation has greatly impacted my ability to carry out work from home.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Connection Problem


In [787]:
import pandas as pd

# Ensure customer_id is the index for merging
activity_data = activity_data.set_index("customer_id") if "customer_id" in activity_data.columns else activity_data
customer_data = customer_data.set_index("customer_id") if "customer_id" in customer_data.columns else customer_data
complaints_data = complaints_data.set_index("customer_id") if "customer_id" in complaints_data.columns else complaints_data

# Merge customer_data with activity_data first
merged_data = customer_data.merge(activity_data, left_index=True, right_index=True, how="left")

# Merge complaints_data with only relevant columns: 'complaint' and 'category'
merged_data = merged_data.merge(
    complaints_data[["complaint", "category"]], 
    left_index=True, 
    right_index=True, 
    how="left"
)

# Create complaint_boolean (1 if a complaint exists, 0 otherwise)
merged_data["complaint_boolean"] = merged_data["complaint"].notna().astype(int)

# Reset index if needed
merged_data.reset_index(inplace=True)

# Display output
print(f"Number of rows in merged dataset: {merged_data.shape[0]}")
print(merged_data.head())

Number of rows in merged dataset: 10000
   customer_id birth_date  join_date  churn_in_3mos      plan_type  plan_pay-as-you-go  plan_postpaid  plan_prepaid  age_at_joining  years_on_plan  data_usage_2021-01-01 00:00:00  data_usage_2021-02-01 00:00:00  data_usage_2021-03-01 00:00:00  data_usage_2021-04-01 00:00:00  data_usage_2021-05-01 00:00:00  data_usage_2021-06-01 00:00:00  data_usage_2021-07-01 00:00:00  data_usage_2021-08-01 00:00:00  data_usage_2021-09-01 00:00:00  data_usage_2021-10-01 00:00:00  data_usage_2021-11-01 00:00:00  data_usage_2021-12-01 00:00:00  phone_usage_2021-01-01 00:00:00  phone_usage_2021-02-01 00:00:00  phone_usage_2021-03-01 00:00:00  phone_usage_2021-04-01 00:00:00  phone_usage_2021-05-01 00:00:00  phone_usage_2021-06-01 00:00:00  phone_usage_2021-07-01 00:00:00  phone_usage_2021-08-01 00:00:00  phone_usage_2021-09-01 00:00:00  phone_usage_2021-10-01 00:00:00  phone_usage_2021-11-01 00:00:00  phone_usage_2021-12-01 00:00:00  use_app_2021-01-01 00:00:00  use

In [788]:
# Count the number of rows
num_rows = merged_data.shape[0]
print(f"Number of rows in activity_pivot: {num_rows}")

Number of rows in activity_pivot: 10000


### Feature Engineering

In [789]:
import numpy as np

# Define conditions for plan_type
conditions = [
    merged_data["plan_pay-as-you-go"] == True,
    merged_data["plan_postpaid"] == True,
    merged_data["plan_prepaid"] == True
]

# Define corresponding plan names
choices = ["pay-as-you-go", "postpaid", "prepaid"]

# Assign values based on conditions
merged_data["plan_type"] = np.select(conditions, choices, default="unknown")  # Default to "unknown" if none are True

# Display output
print(merged_data[["plan_pay-as-you-go", "plan_postpaid", "plan_prepaid", "plan_type"]].head())

   plan_pay-as-you-go  plan_postpaid  plan_prepaid      plan_type
0                   0              1             0       postpaid
1                   1              0             0  pay-as-you-go
2                   0              0             1        prepaid
3                   0              0             1        prepaid
4                   0              0             1        prepaid


In [790]:
columns = merged_data.columns

In [791]:
import re

# Initialize lists
data_usage_cols = []
phone_usage_cols = []
use_app_cols = []

# Regex patterns for each type of usage
patterns = {
    'data_usage': r'^data_usage_',
    'phone_usage': r'^phone_usage_',
    'use_app': r'^use_app_'
}

# Loop through the column names and append to the respective list
for col in columns:
    if re.search(patterns['data_usage'], col):
        data_usage_cols.append(col)
    elif re.search(patterns['phone_usage'], col):
        phone_usage_cols.append(col)
    elif re.search(patterns['use_app'], col):
        use_app_cols.append(col)

# Print the lists to verify
print("Data Usage Columns:", data_usage_cols)
print("Phone Usage Columns:", phone_usage_cols)
print("App Usage Columns:", use_app_cols)

Data Usage Columns: ['data_usage_2021-01-01 00:00:00', 'data_usage_2021-02-01 00:00:00', 'data_usage_2021-03-01 00:00:00', 'data_usage_2021-04-01 00:00:00', 'data_usage_2021-05-01 00:00:00', 'data_usage_2021-06-01 00:00:00', 'data_usage_2021-07-01 00:00:00', 'data_usage_2021-08-01 00:00:00', 'data_usage_2021-09-01 00:00:00', 'data_usage_2021-10-01 00:00:00', 'data_usage_2021-11-01 00:00:00', 'data_usage_2021-12-01 00:00:00']
Phone Usage Columns: ['phone_usage_2021-01-01 00:00:00', 'phone_usage_2021-02-01 00:00:00', 'phone_usage_2021-03-01 00:00:00', 'phone_usage_2021-04-01 00:00:00', 'phone_usage_2021-05-01 00:00:00', 'phone_usage_2021-06-01 00:00:00', 'phone_usage_2021-07-01 00:00:00', 'phone_usage_2021-08-01 00:00:00', 'phone_usage_2021-09-01 00:00:00', 'phone_usage_2021-10-01 00:00:00', 'phone_usage_2021-11-01 00:00:00', 'phone_usage_2021-12-01 00:00:00']
App Usage Columns: ['use_app_2021-01-01 00:00:00', 'use_app_2021-02-01 00:00:00', 'use_app_2021-03-01 00:00:00', 'use_app_2021-04

In [792]:
# Calculate the mean for each row across the specified columns for data, phone usage and app usage
merged_data['monthly_avg_data'] = merged_data[data_usage_cols].mean(axis=1)
merged_data['monthly_avg_phone'] = merged_data[phone_usage_cols].mean(axis=1)
merged_data['monthly_avg_app'] = merged_data[use_app_cols].mean(axis=1)

# Print the DataFrame to verify the new columns
print(merged_data[['monthly_avg_data', 'monthly_avg_phone', 'monthly_avg_app']].head())

   monthly_avg_data  monthly_avg_phone  monthly_avg_app
0         76.915833        3017.853333         0.666667
1         12.286667        1750.020833         0.916667
2         36.716667        2856.807500         0.583333
3         31.731667        2584.227500         0.916667
4         36.001667        2453.307500         0.750000


In [793]:
# Assuming use_app_cols is a list of your monthly app usage columns
merged_data['monthly_avg_app'] = merged_data[use_app_cols].mean(axis=1)

# Define the bins and labels according to the suggested ranges
app_usage_bins = [0.0, 0.4, 0.6, 0.75, 0.9, 1.0]
app_usage_labels = ['very_low', 'low', 'medium', 'high', 'very_high']

# Create the binned column in merged_data
merged_data['app_usage_bins'] = pd.cut(
    merged_data['monthly_avg_app'],
    bins=app_usage_bins,
    labels=app_usage_labels,
    include_lowest=True
)

# Display a quick summary of the binning
binning_summary = merged_data['app_usage_bins'].value_counts().sort_index()
binning_summary

app_usage_bins
very_low       31
low          1585
medium       4509
high         2285
very_high    1590
Name: count, dtype: int64

In [794]:
# Define the bins based on specific cutoffs and create categories

merged_data['phone_usage_bins'] = pd.cut(merged_data['monthly_avg_phone'], 
                                         bins=[0, 2200, 2250, 2300, 2350, 2400, 2450, 2500, float('inf')], 
                                         labels=['lower 2200', '2200-2250', '2250-2300', '2300-2350', '2350-2400', '2400-2450', '2450-2500', 'over 2500'],
                                         right=False) 

In [795]:
merged_data['current_age'] = merged_data['age_at_joining'] + merged_data['years_on_plan']

merged_data['join_month'] = pd.to_datetime(merged_data['join_date']).dt.strftime('%m_%Y')
merged_data['join_quarter'] = 'Q' + pd.to_datetime(merged_data['join_date']).dt.quarter.astype(str) + '_' + pd.to_datetime(merged_data['join_date']).dt.year.astype(str)

In [796]:
from datetime import datetime

# Function to parse the month and year
def parse_month_year(date_string):
    # Ensure the date format matches the example provided
    return datetime.strptime(date_string, '%d/%m/%Y').strftime('%Y-%m')

activity_data_2 = pd.read_csv(path_activity)

# Assuming activity_data is already loaded into the DataFrame
activity_data_2['month_year'] = activity_data_2['month'].apply(parse_month_year)

# Simplify merged_data to include only necessary columns
# Including the 'phone_usage_bins' directly from merged_data
# Also including 'join_month' and 'join_quarter'
merged_data_simplified = merged_data[['customer_id', 'churn_in_3mos', 'category', 'complaint', 'plan_type', 'phone_usage_bins', 'join_month', 'join_quarter', 'monthly_avg_app', 'app_usage_bins']]

# Merge activity_data with merged_data_simplified on customer_id and align other data by using 'left' join
all_usage_long = pd.merge(activity_data_2, merged_data_simplified, on='customer_id', how='left')

# Adjust this DataFrame to handle all data from both sets
all_usage_long['month_year'] = all_usage_long['month'].apply(parse_month_year)  # Overwrite month_year with correct values from activity_data

# Drop the original 'month' column as 'month_year' is now accurate
all_usage_long.drop('month', axis=1, inplace=True)

# Ensure the data structure is correct
print(all_usage_long.head())

   customer_id  data_usage  phone_usage  use_app month_year  churn_in_3mos            category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                complaint      plan_type phone_usage_bins join_month join_quarter  monthly_avg_app app_usage_bins
0        10000       43.61      4570.12        1    2021-01              0                 NaN                                                          

### LLM Pipeline

In [797]:
from transformers import pipeline
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Hugging Face and essential packages are working!")

Hugging Face and essential packages are working!


In [798]:
import os
from huggingface_hub import login

token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    login(token=token)
    print("Logged in successfully!")
else:
    print("Token not found. Make sure it's stored correctly.")

Logged in successfully!


In [799]:
import os
from huggingface_hub import login

token = os.getenv("HUGGINGFACE_TOKEN")

In [800]:
import pandas as pd
import re
import spacy
import time
from collections import Counter, defaultdict

# Load spaCy NLP model (optimized for efficiency)
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def replace_user_ids(text):
    """Replace 5-digit user IDs with [USER ID]"""
    return re.sub(r'\b\d{5}\b', '[USER ID]', text)

def split_into_brackets(text):
    """Split text into sub-phrases (Nebensätze) using common separators, including newlines and periods."""
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return [p.strip() for p in re.split(r'[.,:;\-\n]', text) if p.strip()]

def extract_phrases(merged_data):
    """
    Extract general_phrases and category_phrases based on frequency across categories.
    General phrases appear across multiple categories, while category phrases are unique to one category.
    """
    all_phrases = []
    phrase_category_map = defaultdict(set)
    
    for _, row in merged_data.iterrows():
        category = row['category']
        for phrase in row['bracket_phrases']:
            all_phrases.append(phrase)
            phrase_category_map[phrase].add(category)
    
    phrase_counts = Counter(all_phrases)
    total_complaints = len(merged_data)

    # General phrases appear across multiple categories & are moderately frequent (not too rare, not too common)
    general_phrases = {
        phrase for phrase, cats in phrase_category_map.items() 
        if len(cats) > 1 and 0.02 < phrase_counts[phrase] / total_complaints <= 0.9
    }

    # Category phrases are specific to a single category
    category_phrases = {phrase for phrase, cats in phrase_category_map.items() if len(cats) == 1}

    # Assign extracted phrases back to dataframe
    merged_data['general_phrases'] = merged_data['bracket_phrases'].apply(lambda phrases: [p for p in phrases if p in general_phrases])
    merged_data['category_phrases'] = merged_data['bracket_phrases'].apply(lambda phrases: [p for p in phrases if p in category_phrases])

    return merged_data

def process_complaints(merged_data):
    start_time = time.time()
    
    # Step 1: Keep only complaints
    merged_data = merged_data[merged_data['complaint_boolean'] == True].copy()
    print(f"Step 1: Filtered complaints - {time.time() - start_time:.2f}s")
    
    # Step 2: Replace user IDs
    merged_data['temp_complaint'] = merged_data['complaint'].apply(replace_user_ids)
    print(f"Step 2: Replaced user IDs - {time.time() - start_time:.2f}s")
    
    # Step 3: Extract phrases from brackets
    merged_data['bracket_phrases'] = merged_data['temp_complaint'].apply(split_into_brackets)
    print(f"Step 3: Split into brackets - {time.time() - start_time:.2f}s")
    
    # Step 4: Extract general and category phrases correctly
    merged_data = extract_phrases(merged_data)
    print(f"Step 4: Extracted general and category phrases - {time.time() - start_time:.2f}s")
    
    # Step 5: Drop unnecessary columns
    merged_data.drop(columns=['temp_complaint', 'bracket_phrases'], inplace=True)
    print(f"Step 5: Dropped intermediate columns - {time.time() - start_time:.2f}s")
    
    print(f"Total Processing Time: {time.time() - start_time:.2f}s")
    return merged_data

# Run the processing pipeline
complaint_analysis = process_complaints(merged_data)
complaint_analysis

Step 1: Filtered complaints - 0.01s
Step 2: Replaced user IDs - 0.13s
Step 3: Split into brackets - 0.36s
Step 4: Extracted general and category phrases - 11.41s
Step 5: Dropped intermediate columns - 11.41s
Total Processing Time: 11.41s


,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,data_usage_2021-01-01 00:00:00,data_usage_2021-02-01 00:00:00,data_usage_2021-03-01 00:00:00,data_usage_2021-04-01 00:00:00,data_usage_2021-05-01 00:00:00,data_usage_2021-06-01 00:00:00,data_usage_2021-07-01 00:00:00,data_usage_2021-08-01 00:00:00,data_usage_2021-09-01 00:00:00,data_usage_2021-10-01 00:00:00,data_usage_2021-11-01 00:00:00,data_usage_2021-12-01 00:00:00,phone_usage_2021-01-01 00:00:00,phone_usage_2021-02-01 00:00:00,phone_usage_2021-03-01 00:00:00,phone_usage_2021-04-01 00:00:00,phone_usage_2021-05-01 00:00:00,phone_usage_2021-06-01 00:00:00,phone_usage_2021-07-01 00:00:00,phone_usage_2021-08-01 00:00:00,phone_usage_2021-09-01 00:00:00,phone_usage_2021-10-01 00:00:00,phone_usage_2021-11-01 00:00:00,phone_usage_2021-12-01 00:00:00,use_app_2021-01-01 00:00:00,use_app_2021-02-01 00:00:00,use_app_2021-03-01 00:00:00,use_app_2021-04-01 00:00:00,use_app_2021-05-01 00:00:00,use_app_2021-06-01 00:00:00,use_app_2021-07-01 00:00:00,use_app_2021-08-01 00:00:00,use_app_2021-09-01 00:00:00,use_app_2021-10-01 00:00:00,use_app_2021-11-01 00:00:00,use_app_2021-12-01 00:00:00,complaint,category,complaint_boolean,monthly_avg_data,monthly_avg_phone,monthly_avg_app,app_usage_bins,phone_usage_bins,current_age,join_month,join_quarter,general_phrases,category_phrases
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,2.07,13.87,12.11,3.40,12.22,21.06,4.29,10.17,21.43,8.85,16.34,21.63,2038.61,1952.95,1762.70,1383.54,217.01,174.23,1983.84,799.41,3371.37,2168.73,227.18,4920.68,0,1,1,1,1,1,1,1,1,1,1,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10001, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: There have been frequent interruptions to my service, and I am unable to rely on the connection..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This situation has greatly impacted my ability to carry out work from home.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Service Disruption,1,12.286667,1750.020833,0.916667,very_high,lower 2200,26.975342,01_2015,Q1_2015,[This situation has greatly impacted my ability to carry out work from home],"[There have been frequent interruptions to my service, and I am unable to rely on the connection]"
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,15.28,40.19,27.20,17.89,48.10,41.49,43.77,42.63,42.63,46.94,26.22,39.68,4627.57,1785.52,4617.92,2639.51,2159.29,3285.00,2919.12,4636.01,186.52,243.33,2102.91,236.99,1,1,0,1,1,1,1,1,0,1,0,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10004, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: My internet connection was unstable last week, but after contacting technical support, the issue was resolved..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. The lack of resolution is causing frustration and affecting my usage of the service.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Connection Problem,1,36.001667,2453.307500,0.750000,medium,2450-2500,51.624658,05_2019,Q2_2019,[The lack of resolution is causing frustration and affecting my usage of the service],"[My internet connection was unstable last week, but after contacting technical support, the issue was resolved]"
12,10012,2004-03-16,2018-02-04,1,prepaid,0,0,1,13,3.909589,9.29,16.31,12.54,46.52,18.15,19.19,4.86,30.72,24.73,50.11,23.61,30.55,4895.47,2630.38,4076.20,1984.58,2192.47,557.52,534.59,624.26,3371.39,506.51,231.56,591.07,1,0,1,1,0,1,1,1,1,1,1,1,"Subject: 

In [801]:
import pandas as pd
import shutil

# Convert category_phrases lists into immutable tuples for deduplication
complaint_analysis['category_phrases_tuple'] = complaint_analysis['category_phrases'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# Extract unique rows based on category_phrases and category
unique_category_rows = complaint_analysis[['category', 'category_phrases_tuple']].drop_duplicates().reset_index(drop=True)

# Convert tuples back to lists for better readability in CSV
unique_category_rows['category_phrases'] = unique_category_rows['category_phrases_tuple'].apply(list)

# Assign unique ID
unique_category_rows.insert(0, 'category_phrase_id', range(1, len(unique_category_rows) + 1))

# Drop the temporary tuple column
unique_category_rows.drop(columns=['category_phrases_tuple'], inplace=True)

# Save to CSV
csv_filename = "unique_category_phrases_with_category.csv"
unique_category_rows.to_csv(csv_filename, index=False)

# Move file for download (if applicable)
shutil.move(csv_filename, f"{csv_filename}")

print(f"✅ File saved: {csv_filename} | Ready for download.")

✅ File saved: unique_category_phrases_with_category.csv | Ready for download.


In [803]:
import pandas as pd

# Load the dataset
file_path = "unique_category_phrases_with_category_labelled.xlsx"  # Adjust filename if needed
labels = pd.read_excel(file_path)  # Use pd.read_csv() if it's a CSV file

# Display the first few rows
print(labels.head())

   category_phrase_id       category                                                                                                                                       category_phrases     issue_description  issue_resolved  issue_not_persistent  refund_or_correction_needed  product_related
0                   8  Billing Issue              ['There are charges on my bill that I donâ€™t recognize', 'and customer service has not taken any meaningful steps to resolve the issue']  Customer overcharged           False                 False                         True            False
1                  10  Billing Issue                                 ['I had an issue with my bill last month', 'but customer service quickly resolved the matter and refunded the charge']  Customer overcharged            True                  True                         True            False
2                  18  Billing Issue  ['For the past few months', 'my bills have included unexplained charges', 'I hav

In [816]:
import pandas as pd

# 📌 Load both datasets
labels_file_path = "unique_category_phrases_with_category_labelled.xlsx"  # Adjust filename if needed
labels = pd.read_excel(labels_file_path)  # Use pd.read_csv() if CSV

# 📌 Merge to update `category_phrases` using `category_phrase_id`
labels_updated = labels.merge(
    unique_category_rows[["category_phrase_id", "category_phrases"]],
    on="category_phrase_id",  # Match rows by 'category_phrase_id'
    how="left",  # Keep all rows from labels, update `category_phrases` where possible
    suffixes=("", "_updated")  # Avoid column name conflicts
)

# 📌 Ensure updates are correctly applied
labels_updated["category_phrases"] = labels_updated["category_phrases_updated"].fillna(labels_updated["category_phrases"])
labels_updated.drop(columns=["category_phrases_updated"], inplace=True)  # Clean up extra column

# 📌 Save updated dataset
labels_updated.to_excel("updated_labels.xlsx", index=False)  # Adjust to .csv if needed

print("✅ `category_phrases` successfully updated using `unique_category_rows`!")
print(labels_updated.head())  # Display first few rows for verification

✅ `category_phrases` successfully updated using `unique_category_rows`!
   category_phrase_id       category                                                                                                                               category_phrases     issue_description  issue_resolved  issue_not_persistent  refund_or_correction_needed  product_related
0                   8  Billing Issue            [There are charges on my bill that I don’t recognize, and customer service has not taken any meaningful steps to resolve the issue]  Customer overcharged           False                 False                         True            False
1                  10  Billing Issue                             [I had an issue with my bill last month, but customer service quickly resolved the matter and refunded the charge]  Customer overcharged            True                  True                         True            False
2                  18  Billing Issue  [For the past few months, my bil

In [827]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 📌 Merge similar categories for balanced stratification
def merge_categories(df):
    """Merges similar complaint categories for better training balance."""
    df['merged_category'] = df['category'].replace({
        "Billing Issue": "Billing & Charges",
        "Unauthorized Charges": "Billing & Charges",
        "Service Disruption": "Technical Issues",
        "Connection Problem": "Technical Issues"
    })
    return df

# 📌 Function for Train-Test Split
def split_dataset(df, test_size=0.2):
    """Splits dataset into train and test sets, ensuring category representation."""
    df = merge_categories(df)  # Ensure categories are merged before splitting
    
    train_df, test_df = train_test_split(
        df, 
        test_size=test_size, 
        stratify=df['merged_category'], 
        random_state=42
    )

    print("\n✅ Train-test split complete. Category distributions:")
    print("\nTraining Set:\n", train_df['merged_category'].value_counts())
    print("\nTest Set:\n", test_df['merged_category'].value_counts())

    train_df.to_csv("train_dataset.csv", index=False)
    test_df.to_csv("test_dataset.csv", index=False)
    return train_df, test_df

# ✅ Run the train-test split
train_labels, test_labels = split_dataset(labels)


✅ Train-test split complete. Category distributions:

Training Set:
 merged_category
Billing & Charges    24
Technical Issues     24
Promotional Issue     8
Name: count, dtype: int64

Test Set:
 merged_category
Billing & Charges    6
Technical Issues     6
Promotional Issue    2
Name: count, dtype: int64


In [828]:
import time
import re
import spacy
import pandas as pd
from transformers import pipeline

# ✅ Load Models
print("🔄 Loading models...")
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")
nlp = spacy.load("en_core_web_sm")

# 🎯 Define Features & Prompts
feature_prompts = {
    "issue_resolved": "Was the issue resolved?",
    "product_related": "Does the user have bad connection or interruptions?",
    "refund_or_correction_needed": "Is the customer charged or billed wrongly?"
}

# 📌 Function to detect issue persistence using SpaCy
def detect_past_tense(text):
    """Detects whether an issue is described in the past, assuming persistence otherwise."""
    doc = nlp(text)

    # Check for "has been" or "have been" (indicates ongoing issue)
    if re.search(r"has\s?.?been|have\s?.?been", text.lower()):
        return False  # Persistent issue

    # Count past tense verbs
    past_count = sum(1 for token in doc if token.tag_ in ["VBD", "VBN"])
    total_verbs = sum(1 for token in doc if token.pos_ == "VERB")

    # Classify based on verb tense ratio
    return past_count / total_verbs > 0.5 if total_verbs > 0 else False

# 📌 Function to extract features only for **unique phrases**
def extract_features(df):
    """Extracts features efficiently by processing unique phrases only."""
    start_time = time.time()
    
    # ✅ Get unique phrases
    unique_phrases = df["category_phrases"].dropna().unique()
    results = {}

    for text in unique_phrases:
        feature_values = {}

        # ✅ Extract LLM-based features
        for feature, prompt in feature_prompts.items():
            input_prompt = f"{prompt} Only answer 'Yes' or 'No'. Context: {text}"
            try:
                response = llm_pipeline(input_prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
                feature_values[feature] = response in ["yes", "true"]  # Convert to boolean
            except Exception as e:
                feature_values[feature] = None  # Handle errors

        # ✅ Extract SpaCy-based feature
        feature_values["issue_not_persistent"] = detect_past_tense(text)

        # ✅ Store results for this phrase
        results[text] = feature_values

    print(f"\n✅ Feature extraction completed in {time.time() - start_time:.2f}s 🚀")
    
    # Convert to DataFrame
    features_df = pd.DataFrame.from_dict(results, orient="index").reset_index()
    features_df.rename(columns={"index": "category_phrases"}, inplace=True)
    
    return features_df

🔄 Loading models...


Device set to use cpu


In [829]:
import pickle

# 📌 Train and Save Model (Only on Unique Phrases)
def train_and_save_model(df, model_filename="complaint_analysis_model.pkl"):
    """Extracts features on unique phrases and saves them as a model."""
    features_df = extract_features(df)
    
    # Save model (features) using Pickle
    with open(model_filename, "wb") as file:
        pickle.dump(features_df, file)
    
    print(f"✅ Model saved as {model_filename}")

# ✅ Train and Save the Model
print("🔄 Training the model on unique phrases in training set...")
train_and_save_model(train_labels)

🔄 Training the model on unique phrases in training set...

✅ Feature extraction completed in 70.70s 🚀
✅ Model saved as complaint_analysis_model.pkl


In [821]:
import pickle
import pandas as pd

# 🔹 Path to your saved pickle file (update if necessary)
pickle_file_path = "complaint_analysis_model.pkl"  # Adjust if in a different directory

# 🔹 Load the pickle file
try:
    with open(pickle_file_path, "rb") as file:
        trained_features = pickle.load(file)

    # 🔎 Check the type of the loaded object
    print(f"✅ Loaded object type: {type(trained_features)}")

    # 🔎 If it's a DataFrame, display basic info
    if isinstance(trained_features, pd.DataFrame):
        print("\n📌 First few rows of `trained_features`:")
        display(trained_features.head())  # Use `print(trained_features.head())` if outside Jupyter

        print("\n📌 DataFrame Info:")
        print(trained_features.info())

        print("\n📌 Unique Phrases Stored:")
        print(trained_features["category_phrases"].nunique())

    else:
        print("\n⚠️ Warning: The pickle file does not contain a DataFrame. Here’s its raw content:")
        print(trained_features)

except Exception as e:
    print(f"⚠️ Error loading pickle file: {e}")

✅ Loaded object type: <class 'pandas.core.frame.DataFrame'>

📌 First few rows of `trained_features`:


,category_phrases,issue_resolved,product_related,refund_or_correction_needed,issue_not_persistent
0,"['There was an unauthorized charge on my bill', 'but customer support refunded the amount quickly']",True,False,True,True
1,"['The internet connection has been cutting out for several weeks', 'and customer service has not provided a solution']",False,True,False,False
2,['There was an unannounced outage that left me without service for almost an entire day'],False,True,False,True
3,"['Over the past few months', 'I have been overcharged for services that were not part of my plan', 'Customer support has failed to address this']",False,True,True,False
4,"['The support team restored the service quickly after a brief outage', 'and I appreciate the swift response']",True,False,False,False



📌 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   category_phrases             56 non-null     object
 1   issue_resolved               56 non-null     bool  
 2   product_related              56 non-null     bool  
 3   refund_or_correction_needed  56 non-null     bool  
 4   issue_not_persistent         56 non-null     bool  
dtypes: bool(4), object(1)
memory usage: 804.0+ bytes
None

📌 Unique Phrases Stored:
56


In [826]:
def apply_saved_model(new_df, model_filename="complaint_analysis_model.pkl"):
    """Loads a saved model and applies it to a new dataset efficiently, ensuring all unique phrases are processed."""
    
    # ✅ Load existing trained features
    with open(model_filename, "rb") as file:
        trained_features = pickle.load(file)

    # ✅ Ensure category_phrases are properly cleaned and converted to plain strings
    def clean_text(text):
        """Converts lists into plain text strings and ensures proper formatting."""
        if isinstance(text, list):
            return " ".join(map(str, text))  # Join list elements into a single string
        return str(text).strip().lower()

    trained_features["category_phrases"] = trained_features["category_phrases"].apply(clean_text)

    # ✅ Ensure `category_phrases` in `new_df` are cleaned in the same way
    new_df["category_phrases"] = new_df["category_phrases"].apply(clean_text)
    
    unique_phrases_new = new_df["category_phrases"].dropna().drop_duplicates()

    # ✅ Extract known phrases from trained model
    known_phrases = set(trained_features["category_phrases"].dropna().drop_duplicates())

    print(f"🔎 Total unique phrases in new dataset: {len(unique_phrases_new)}")
    print(f"🔎 Total known phrases in trained model: {len(known_phrases)}")

    # ✅ Identify truly new phrases
    new_phrases_to_process = unique_phrases_new[~unique_phrases_new.isin(known_phrases)]

    print(f"🛠 New phrases detected for processing: {len(new_phrases_to_process)}")

    if not new_phrases_to_process.empty:
        print(f"🔄 Processing {len(new_phrases_to_process)} new unique phrases...")
        new_features = extract_features(pd.DataFrame({"category_phrases": new_phrases_to_process.tolist()}))
        
        # ✅ Append new phrases to stored model
        trained_features = pd.concat([trained_features, new_features], ignore_index=True)

        # ✅ Save the updated model with new phrases
        with open(model_filename, "wb") as file:
            pickle.dump(trained_features, file)
        
        print(f"✅ Model updated with {len(new_phrases_to_process)} new phrases.")

    # ✅ Merge predictions back to the full dataset, **removing duplicate columns**
    new_df = new_df.merge(trained_features, on="category_phrases", how="left")

    # 🔥 **Fix Duplicate Columns** (Remove `_x` and `_y`)
    for col in trained_features.columns:
        if f"{col}_x" in new_df.columns and f"{col}_y" in new_df.columns:
            # Keep the non-null values and drop duplicate columns
            new_df[col] = new_df[f"{col}_x"].combine_first(new_df[f"{col}_y"])
            new_df.drop(columns=[f"{col}_x", f"{col}_y"], inplace=True)

    print("✅ Model applied to new dataset!")
    return new_df

In [825]:
complaint_analysis

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,data_usage_2021-01-01 00:00:00,data_usage_2021-02-01 00:00:00,data_usage_2021-03-01 00:00:00,data_usage_2021-04-01 00:00:00,data_usage_2021-05-01 00:00:00,data_usage_2021-06-01 00:00:00,data_usage_2021-07-01 00:00:00,data_usage_2021-08-01 00:00:00,data_usage_2021-09-01 00:00:00,data_usage_2021-10-01 00:00:00,data_usage_2021-11-01 00:00:00,data_usage_2021-12-01 00:00:00,phone_usage_2021-01-01 00:00:00,phone_usage_2021-02-01 00:00:00,phone_usage_2021-03-01 00:00:00,phone_usage_2021-04-01 00:00:00,phone_usage_2021-05-01 00:00:00,phone_usage_2021-06-01 00:00:00,phone_usage_2021-07-01 00:00:00,phone_usage_2021-08-01 00:00:00,phone_usage_2021-09-01 00:00:00,phone_usage_2021-10-01 00:00:00,phone_usage_2021-11-01 00:00:00,phone_usage_2021-12-01 00:00:00,use_app_2021-01-01 00:00:00,use_app_2021-02-01 00:00:00,use_app_2021-03-01 00:00:00,use_app_2021-04-01 00:00:00,use_app_2021-05-01 00:00:00,use_app_2021-06-01 00:00:00,use_app_2021-07-01 00:00:00,use_app_2021-08-01 00:00:00,use_app_2021-09-01 00:00:00,use_app_2021-10-01 00:00:00,use_app_2021-11-01 00:00:00,use_app_2021-12-01 00:00:00,complaint,category,complaint_boolean,monthly_avg_data,monthly_avg_phone,monthly_avg_app,app_usage_bins,phone_usage_bins,current_age,join_month,join_quarter,general_phrases,category_phrases,category_phrases_tuple,issue_resolved_x,product_related_x,refund_or_correction_needed_x,issue_not_persistent_x,issue_resolved_y,product_related_y,refund_or_correction_needed_y,issue_not_persistent_y
0,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,2.07,13.87,12.11,3.40,12.22,21.06,4.29,10.17,21.43,8.85,16.34,21.63,2038.61,1952.95,1762.70,1383.54,217.01,174.23,1983.84,799.41,3371.37,2168.73,227.18,4920.68,0,1,1,1,1,1,1,1,1,1,1,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10001, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: There have been frequent interruptions to my service, and I am unable to rely on the connection..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This situation has greatly impacted my ability to carry out work from home.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Service Disruption,1,12.286667,1750.020833,0.916667,very_high,lower 2200,26.975342,01_2015,Q1_2015,[This situation has greatly impacted my ability to carry out work from home],there have been frequent interruptions to my service and i am unable to rely on the connection,"(There have been frequent interruptions to my service, and I am unable to rely on the connection)",False,True,False,False,False,True,False,False
1,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,15.28,40.19,27.20,17.89,48.10,41.49,43.77,42.63,42.63,46.94,26.22,39.68,4627.57,1785.52,4617.92,2639.51,2159.29,3285.00,2919.12,4636.01,186.52,243.33,2102.91,236.99,1,1,0,1,1,1,1,1,0,1,0,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10004, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: My internet connection was unstable last week, but after contacting technical support, the issue was resolved..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. The lack of resolution is causing frustration and affecting my usage of the service.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Connection Problem,1,36.001667,2453.307500,0.750000,medium,2450-2500,51.624658,05_2019,Q2_2019,[The lack of resolution is causing frustration and affecting my usage of the service],my internet connection was 

In [755]:
import time
import re
import spacy
import pickle
import pandas as pd
from transformers import pipeline
from sklearn.model_selection import train_test_split

# ✅ Load Models
print("🔄 Loading models...")
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")
nlp = spacy.load("en_core_web_sm")

# 🎯 Define Features & Prompts
feature_prompts = {
    "issue_resolved": "Was the issue resolved?",
    "product_related": "Does the user have bad connection or interruptions?",
    "refund_or_correction_needed": "Is the customer charged or billed wrongly?"
}

# 📌 Function to detect issue persistence using SpaCy
def detect_past_tense(text):
    """Detects whether an issue is described in the past, assuming persistence otherwise."""
    doc = nlp(text)

    # Check for "has been" or "have been" (indicates ongoing issue)
    if re.search(r"has\s?.?been|have\s?.?been", text.lower()):
        return False  # Persistent issue

    # Count past tense verbs
    past_count = sum(1 for token in doc if token.tag_ in ["VBD", "VBN"])
    total_verbs = sum(1 for token in doc if token.pos_ == "VERB")

    # Classify based on verb tense ratio
    return past_count / total_verbs > 0.5 if total_verbs > 0 else False

# 📌 Function to extract features only for **unique phrases**
def extract_features(df):
    """Extracts features efficiently by processing unique phrases only."""
    start_time = time.time()
    
    # ✅ Get unique phrases
    unique_phrases = df["category_phrases"].dropna().unique()
    results = {}

    for text in unique_phrases:
        feature_values = {}

        # ✅ Extract LLM-based features
        for feature, prompt in feature_prompts.items():
            input_prompt = f"{prompt} Only answer 'Yes' or 'No'. Context: {text}"
            try:
                response = llm_pipeline(input_prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
                feature_values[feature] = response in ["yes", "true"]  # Convert to boolean
            except Exception as e:
                feature_values[feature] = None  # Handle errors

        # ✅ Extract SpaCy-based feature
        feature_values["issue_not_persistent"] = detect_past_tense(text)

        # ✅ Store results for this phrase
        results[text] = feature_values

    print(f"\n✅ Feature extraction completed in {time.time() - start_time:.2f}s 🚀")
    
    # Convert to DataFrame
    features_df = pd.DataFrame.from_dict(results, orient="index").reset_index()
    features_df.rename(columns={"index": "category_phrases"}, inplace=True)
    
    return features_df

# 📌 Merge similar categories
def merge_categories(df):
    """Merges similar complaint categories for better training balance."""
    df['merged_category'] = df['category'].replace({
        "Billing Issue": "Billing & Charges",
        "Unauthorized Charges": "Billing & Charges",
        "Service Disruption": "Technical Issues",
        "Connection Problem": "Technical Issues"
    })
    return df

# 📌 Train-test split function
def split_dataset(df, test_size=0.2):
    """Splits dataset into train and test sets, ensuring category representation."""
    df = merge_categories(df)  # Ensure categories are merged before splitting
    
    train_df, test_df = train_test_split(
        df, 
        test_size=test_size, 
        stratify=df['merged_category'], 
        random_state=42
    )

    print("\n✅ Train-test split complete. Category distributions:")
    print("\nTraining Set:\n", train_df['merged_category'].value_counts())
    print("\nTest Set:\n", test_df['merged_category'].value_counts())

    train_df.to_csv("train_dataset.csv", index=False)
    test_df.to_csv("test_dataset.csv", index=False)
    return train_df, test_df

# 📌 Train and Save Model (Only on Unique Phrases)
def train_and_save_model(df, model_filename="complaint_analysis_model.pkl"):
    """Extracts features on unique phrases and saves them as a model."""
    features_df = extract_features(df)
    
    # Save model (features) using Pickle
    with open(model_filename, "wb") as file:
        pickle.dump(features_df, file)
    
    print(f"✅ Model saved as {model_filename}")

# 📌 Apply Model to New Data (Ensuring Unique Phrases)
def apply_saved_model(new_df, model_filename="complaint_analysis_model.pkl"):
    """Loads a saved model and applies it to a new dataset efficiently."""
    with open(model_filename, "rb") as file:
        trained_features = pickle.load(file)

    # ✅ Extract features only for **new unique phrases** that aren't already in the model
    unique_phrases_new = new_df["category_phrases"].dropna().unique()
    known_phrases = trained_features["category_phrases"].tolist()
    new_phrases_to_process = [phrase for phrase in unique_phrases_new if phrase not in known_phrases]

    if new_phrases_to_process:
        print(f"🔄 Processing {len(new_phrases_to_process)} new unique phrases...")
        new_features = extract_features(pd.DataFrame({"category_phrases": new_phrases_to_process}))
        
        # ✅ Append new phrases to stored model
        trained_features = pd.concat([trained_features, new_features], ignore_index=True)

        # ✅ Save the updated model
        with open(model_filename, "wb") as file:
            pickle.dump(trained_features, file)
        
        print(f"✅ Model updated with new phrases.")

    # ✅ Merge predictions back to the full dataset (using 'category_phrases' as key)
    new_df = new_df.merge(trained_features, on="category_phrases", how="left")

    print("✅ Model applied to new dataset!")
    return new_df

# 📌 Run Full Process
print("🔄 Splitting dataset...")
train_labels, test_labels = split_dataset(labels)

print("🔄 Training the model on unique phrases in training set...")
train_and_save_model(train_labels)

print("🔄 Applying model to test dataset...")
test_labels = apply_saved_model(test_labels)

# 📌 Save Test Predictions
test_labels.to_csv("test_dataset_with_predictions.csv", index=False)
print("✅ Test predictions saved!")

# 📌 Apply to New Complaints (Ensuring Unique Processing)
print("🔄 Applying model to new dataset...")
complaint_analysis = apply_saved_model(complaint_analysis)

🔄 Loading models...


Device set to use cpu


🔄 Splitting dataset...

✅ Train-test split complete. Category distributions:

Training Set:
 merged_category
Billing & Charges    24
Technical Issues     24
Promotional Issue     8
Name: count, dtype: int64

Test Set:
 merged_category
Billing & Charges    6
Technical Issues     6
Promotional Issue    2
Name: count, dtype: int64
🔄 Training the model on unique phrases in training set...

✅ Feature extraction completed in 82.18s 🚀
✅ Model saved as complaint_analysis_model.pkl
🔄 Applying model to test dataset...
🔄 Processing 14 new unique phrases...

✅ Feature extraction completed in 15.44s 🚀
✅ Model updated with new phrases.
✅ Model applied to new dataset!
✅ Test predictions saved!
🔄 Applying model to new dataset...


TypeError: unhashable type: 'list'

In [717]:
from sklearn.model_selection import train_test_split

# First, merge similar categories
labels['merged_category'] = labels['category'].replace({
    "Billing Issue": "Billing & Charges",
    "Unauthorized Charges": "Billing & Charges",
    "Service Disruption": "Technical Issues",
    "Connection Problem": "Technical Issues"
})

# Check new category distribution
print(labels['merged_category'].value_counts())

# Split ensuring all categories are represented
train_labels, test_labels = train_test_split(
    labels, 
    test_size=0.2, 
    stratify=labels['merged_category'], 
    random_state=42
)

# Check final distributions
print("Training Set:")
print(train_labels['merged_category'].value_counts())

print("\nTest Set:")
print(test_labels['merged_category'].value_counts())

# Save files for LLM training
train_labels.to_csv("train_dataset.csv", index=False)
test_labels.to_csv("test_dataset.csv", index=False)

print("✅ Train-test split complete. Ready for LLM training!")

merged_category
Billing & Charges    30
Technical Issues     30
Promotional Issue    10
Name: count, dtype: int64
Training Set:
merged_category
Billing & Charges    24
Technical Issues     24
Promotional Issue     8
Name: count, dtype: int64

Test Set:
merged_category
Billing & Charges    6
Technical Issues     6
Promotional Issue    2
Name: count, dtype: int64
✅ Train-test split complete. Ready for LLM training!


In [725]:
import time
from transformers import pipeline

# 📌 Load LLM pipeline
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# 🎯 Define features and corresponding prompts
feature_prompts = {
    "issue_resolved": "Was the issue resolved?",
    "product_related": "Does the user have bad connection or interruptions?",
    "refund_or_correction_needed": "Is the customer charged or billed wrongly?"
}

# 🔄 Quick Function to Test Model on Sample Data
def test_llm_on_sample(df, sample_size=10):
    start_time = time.time()
    
    # 📌 Sample Data
    sample = df.sample(n=sample_size, random_state=42)

    # 🚀 Loop Over Each Feature
    for feature, prompt in feature_prompts.items():
        print(f"\n🔍 Checking Predictions for Feature: {feature.upper()}")

        for _, row in sample.iterrows():
            text = row["category_phrases"]
            input_prompt = f"{prompt} Only answer 'Yes' or 'No'. Context: {text}"
            
            try:
                response = llm_pipeline(input_prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
                print(f"📝 Complaint: {text[:80]}... \n  - Predicted: {response} | Actual: {row[feature]}")
            except Exception as e:
                print(f"⚠️ Error processing: {text[:80]}... | Error: {e}")

    print(f"\n✅ Quick LLM Check Completed in {time.time() - start_time:.2f}s 🚀")

# 🔥 Run Quick Check
test_llm_on_sample(labels)  # Assuming 'labels' is your dataset

Device set to use cpu



🔍 Checking Predictions for Feature: ISSUE_RESOLVED
📝 Complaint: ['Frequent disconnections and slow speeds are making it impossible to work from ... 
  - Predicted: no | Actual: False
📝 Complaint: ['There are charges on my bill that I donâ€™t recognize', 'and customer service ... 
  - Predicted: no | Actual: False
📝 Complaint: ['I experienced a brief disruption to my phone service']... 
  - Predicted: no | Actual: False
📝 Complaint: ['A billing discrepancy was identified', 'but the support team resolved it quick... 
  - Predicted: yes | Actual: True
📝 Complaint: ['My internet service has been cut off multiple times', 'leaving me without acce... 
  - Predicted: no | Actual: False
📝 Complaint: ['My internet connection has been unreliable', 'frequently dropping at critical ... 
  - Predicted: no | Actual: False
📝 Complaint: ['There was a billing error', 'but customer service corrected it promptly']... 
  - Predicted: yes | Actual: True
📝 Complaint: ['The discount I was offered when I sign

In [724]:
import time
import re
import spacy

# Load SpaCy NLP model
nlp = spacy.load("en_core_web_sm")

def detect_past_tense(text):
    """
    Detects whether an issue is described in the past.
    If 'has been' or 'have been' (with any character in between) appears, it's persistent.
    Otherwise, checks past-tense verb dominance.
    """
    doc = nlp(text)
    
    # Check for "has been" or "have been" with flexible spacing/punctuation
    if re.search(r"has\s?.?been|have\s?.?been", text.lower()):
        return False  # ✅ Persistent issue (issue_not_persistent = False)

    # Count past tense verbs
    past_count = sum(1 for token in doc if token.tag_ in ["VBD", "VBN"])
    total_verbs = sum(1 for token in doc if token.pos_ == "VERB")

    # If past tense verbs dominate, assume issue is **not persistent**
    return past_count / total_verbs > 0.5 if total_verbs > 0 else False

def check_issue_persistence(df, sample_size=10):
    """Runs issue persistence detection on a sample from the dataset."""
    start_time = time.time()
    sample = df.sample(n=sample_size, random_state=42)

    print("\n🔍 Checking Issue Persistence Predictions:")
    
    for _, row in sample.iterrows():
        text = row["category_phrases"]
        predicted = detect_past_tense(text)  # Run SpaCy-based persistence detection
        actual = row["issue_not_persistent"]  # Ground truth label

        print(f"📝 Complaint: {text[:80]}... \n  - Predicted: {predicted} | Actual: {actual}")

    print(f"\n✅ Quick Persistence Check Completed in {time.time() - start_time:.2f}s 🚀")

# 🔥 Run Quick Check
check_issue_persistence(labels)  # Assuming 'labels' is your dataset


🔍 Checking Issue Persistence Predictions:
📝 Complaint: ['Frequent disconnections and slow speeds are making it impossible to work from ... 
  - Predicted: False | Actual: False
📝 Complaint: ['There are charges on my bill that I donâ€™t recognize', 'and customer service ... 
  - Predicted: False | Actual: False
📝 Complaint: ['I experienced a brief disruption to my phone service']... 
  - Predicted: True | Actual: True
📝 Complaint: ['A billing discrepancy was identified', 'but the support team resolved it quick... 
  - Predicted: True | Actual: True
📝 Complaint: ['My internet service has been cut off multiple times', 'leaving me without acce... 
  - Predicted: False | Actual: False
📝 Complaint: ['My internet connection has been unreliable', 'frequently dropping at critical ... 
  - Predicted: False | Actual: False
📝 Complaint: ['There was a billing error', 'but customer service corrected it promptly']... 
  - Predicted: True | Actual: True
📝 Complaint: ['The discount I was offered when 

In [666]:
def detect_past_tense(text):
    """Checks if the majority of verbs in the sentence are in past tense."""
    doc = nlp(text)
    past_count = sum(1 for token in doc if token.tag_ in ["VBD", "VBN"])  # Past tense verbs
    total_verbs = sum(1 for token in doc if token.pos_ == "VERB")

    return past_count / total_verbs > 0.5 if total_verbs > 0 else False

In [668]:
import spacy

# Load SpaCy NLP model
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def detect_past_tense(text):
    """Checks if the majority of verbs in the sentence are in past tense."""
    doc = nlp(text)
    past_count = sum(1 for token in doc if token.tag_ in ["VBD", "VBN"])  # Past tense verbs
    total_verbs = sum(1 for token in doc if token.pos_ == "VERB")
    
    return past_count / total_verbs > 0.5 if total_verbs > 0 else False

def evaluate_persistence_detection(dataset):
    """Checks per row if the persistence detection was correct and prints results."""
    correct = 0
    total = 0

    print("\n🔍 **Evaluating Issue Persistence Detection**")
    
    for _, row in dataset.iterrows():
        text = row["category_phrases"]
        actual_label = row["issue_persists"]  # True = persists, False = resolved
        predicted_label = not detect_past_tense(text)  # Inverting since past means NOT persistent

        # Compare results
        is_correct = actual_label == predicted_label
        correct += is_correct
        total += 1

        print(f"\n📌 Complaint: {text}")
        print(f"🔹 Actual Persistence: {actual_label}")
        print(f"🔹 Predicted Persistence: {predicted_label}")
        print(f"✅ Correct: {is_correct}")

    accuracy = correct / total * 100
    print(f"\n✅ **Evaluation Complete. Accuracy: {accuracy:.2f}%**")

# Run on train and test sets separately
evaluate_persistence_detection(train_labels)
evaluate_persistence_detection(test_labels)


🔍 **Evaluating Issue Persistence Detection**

📌 Complaint: ['There was an unauthorized charge on my bill', 'but customer support refunded the amount quickly']
🔹 Actual Persistence: False
🔹 Predicted Persistence: False
✅ Correct: True

📌 Complaint: ['The internet connection has been cutting out for several weeks', 'and customer service has not provided a solution']
🔹 Actual Persistence: False
🔹 Predicted Persistence: False
✅ Correct: True

📌 Complaint: ['There was an unannounced outage that left me without service for almost an entire day']
🔹 Actual Persistence: False
🔹 Predicted Persistence: False
✅ Correct: True

📌 Complaint: ['Over the past few months', 'I have been overcharged for services that were not part of my plan', 'Customer support has failed to address this']
🔹 Actual Persistence: False
🔹 Predicted Persistence: False
✅ Correct: True

📌 Complaint: ['The support team restored the service quickly after a brief outage', 'and I appreciate the swift response']
🔹 Actual Persistenc

In [751]:
labels['Severity_Label'].value_counts()

Severity_Label
Neutral          35
Negative         24
Very Negative    11
Name: count, dtype: int64

In [749]:
labels['toxicity_score'].max()

0.0008130717324092984

In [741]:
labels[labels['sentiment_score']==1]

,category_phrase_id,category,category_phrases,issue_description,issue_resolved,issue_not_persistent,refund_or_correction_needed,product_related,merged_category,issue_summary,issue_severity,severe_case,sentiment_score


In [734]:
severe_results

{"['Frequent disconnections and slow speeds are making it impossible to work from home']": False,
 "['There are charges on my bill that I donâ€™t recognize', 'and customer service has not taken any meaningful steps to resolve the issue']": False,
 "['I experienced a brief disruption to my phone service']": False,
 "['A billing discrepancy was identified', 'but the support team resolved it quickly']": False,
 "['My internet service has been cut off multiple times', 'leaving me without access for hours at a time']": False,
 "['My internet connection has been unreliable', 'frequently dropping at critical moments']": False,
 "['There was a billing error', 'but customer service corrected it promptly']": False,
 "['The discount I was offered when I signed up has never been applied', 'This issue remains unresolved']": False,
 "['Iâ€™ve experienced several service interruptions in the past few weeks', 'making it difficult to rely on your service']": False,
 "['Every month', 'I see erroneous ch

In [ ]:
# Extract unique complaint phrases
train_phrases = train_labels["category_phrases"].dropna().tolist()
test_phrases = test_labels["category_phrases"].dropna().tolist()

# Run persistence detection
start_time = time.time()
train_persistence = check_issue_persistence(train_phrases)
test_persistence = check_issue_persistence(test_phrases)
end_time = time.time()

# Assign results back to dataset
train_labels["issue_persists"] = train_labels["category_phrases"].apply(
    lambda phrases: train_persistence.get(phrases, False) if isinstance(phrases, str) else False
)
test_labels["issue_persists"] = test_labels["category_phrases"].apply(
    lambda phrases: test_persistence.get(phrases, False) if isinstance(phrases, str) else False
)

# Save updated datasets
train_labels.to_csv("train_dataset_updated.csv", index=False)
test_labels.to_csv("test_dataset_updated.csv", index=False)

print(f"✅ Issue Persistence Detection Complete in {end_time - start_time:.2f}s")
print("🚀 Updated datasets saved as train_dataset_updated.csv & test_dataset_updated.csv")


In [550]:
import time
from transformers import pipeline

# 🚀 This function determines whether a customer issue was resolved based on complaint descriptions.
# 🔍 It processes unique complaint phrases using a lightweight LLM and assigns a resolution status.

def detect_issue_resolution(category_phrases):
    llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")
    return {phrase: llm_pipeline(f"Was the issue resolved? Only answer 'Yes' or 'No'. Context: {phrase}",
                                 max_length=5, do_sample=False)[0]['generated_text'].strip().lower().startswith("yes")
            for phrase in category_phrases}

# ---- RUN THE PIPELINE ----
start_time = time.time()
resolution_scores = detect_issue_resolution(set(" ".join(phrases) for phrases in complaint_analysis['category_phrases'] if phrases))
complaint_analysis['issue_resolved'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: resolution_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)
print(f"✅ Issue Resolution Detection successful in {time.time() - start_time:.2f}s")

Device set to use cpu


✅ Issue Resolution Detection successful in 27.74s


In [589]:
import time
from transformers import pipeline
import pandas as pd

# Load LLM pipeline
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Extract a sample of 100 rows
sample_df = complaint_analysis.sample(n=100)[['category_phrases']].copy()
sample_df['category_phrases'] = sample_df['category_phrases'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

# 🚀 **Step 1: Issue Resolution Detection (Using Old Engine)**
def detect_issue_resolution(phrases):
    """Detects if an issue was resolved."""
    return {
        phrase: llm_pipeline(f"Was the issue resolved? Answer 'Yes' or 'No'. Context: {phrase}",
                             max_length=5, do_sample=False)[0]['generated_text'].strip().lower().startswith("yes")
        for phrase in phrases
    }

start_time = time.time()
unique_phrases = sample_df['category_phrases'].unique()
resolution_scores = detect_issue_resolution(unique_phrases)
sample_df['issue_resolved'] = sample_df['category_phrases'].map(resolution_scores).fillna(False)
print(f"✅ Issue Resolution Detection completed in {time.time() - start_time:.2f}s")

# 🚀 **Step 2: Ongoing Issue Detection (Only for Unresolved Cases)**
def detect_ongoing_issue(phrases):
    """Detects if an unresolved issue is still ongoing."""
    ongoing_scores = {}

    for phrase in phrases:
        if not resolution_scores.get(phrase, False):  # Only process unresolved issues
            response = llm_pipeline(
                f"Does the customer clearly describe the issue as already happened without continuation? If so, say 'No', otherwise 'Yes': {phrase}",
                max_length=5, do_sample=False
            )[0]['generated_text'].strip().lower()

            ongoing_scores[phrase] = response.startswith("yes")

    return ongoing_scores

start_time = time.time()
ongoing_scores = detect_ongoing_issue(unique_phrases)
sample_df['ongoing_issue'] = sample_df['category_phrases'].map(ongoing_scores).fillna(False)
print(f"✅ Ongoing Issue Detection completed in {time.time() - start_time:.2f}s")

# 🚀 **Step 3: Customer Demand Detection (Only for Unresolved Cases)**
def no_customer_demand(phrases):
    """Detects if a customer is demanding action."""
    demand_scores = {}

    for phrase in phrases:
        if not resolution_scores.get(phrase, False):  # Only process unresolved issues
            response = llm_pipeline(
                f"Does the customer need a refund? If so, say 'Yes', otherwise 'No'. Context: {phrase}",
                max_length=5, do_sample=False
            )[0]['generated_text'].strip().lower()

            demand_scores[phrase] = response.startswith("yes")

    return demand_scores
    

start_time = time.time()
demand_scores = no_customer_demand(unique_phrases)
sample_df['no_customer_demand'] = sample_df['category_phrases'].map(demand_scores).fillna(False)
print(f"✅ Customer Demand Detection completed in {time.time() - start_time:.2f}s")

# 🚀 **Display Results**
sample_df[['category_phrases', 'issue_resolved', 'ongoing_issue', 'no_customer_demand']]

Device set to use cpu


✅ Issue Resolution Detection completed in 20.46s


C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\2453227979.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sample_df['ongoing_issue'] = sample_df['category_phrases'].map(ongoing_scores).fillna(False)


✅ Ongoing Issue Detection completed in 15.04s
✅ Customer Demand Detection completed in 15.39s


C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\2453227979.py:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sample_df['no_customer_demand'] = sample_df['category_phrases'].map(demand_scores).fillna(False)


,category_phrases,issue_resolved,ongoing_issue,no_customer_demand
2707,An incorrect charge appeared on my bill but the customer service team resolved the issue and processed a refund,True,False,False
8819,There was an unannounced outage that left me without service for almost an entire day,False,True,False
2399,The support team restored the service quickly after a brief outage and I appreciate the swift response,True,False,False
9485,There was an issue with the internet speed but support promptly addressed the problem and restored the service,True,False,False
6051,I was charged incorrectly but customer support issued a refund and the issue was resolved,True,False,False
...,...,...,...,...
874,An overcharge appeared on my bill but the issue was handled efficiently and the refund was processed within 24 hours,True,False,False
4460,The internet service was down briefly but the issue was resolved promptly by customer support,True,False,False
4094,There was a service disruption last week but the support team restored the connection quickly,True,False,False
3465,I was charged incorrectly but customer support issued a refund and the issue was resolved,True,False,False


In [571]:
import time
from transformers import pipeline
import pandas as pd

# Load LLM pipeline
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Function to check if an issue is resolved
def detect_issue_resolution(category_phrases):
    """Classifies whether an issue was resolved."""
    resolution_scores = {}

    for phrase in category_phrases:
        prompt = f"Was the issue resolved? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            resolution_scores[phrase] = response.startswith("yes")  # True if "Yes", False otherwise
        except Exception:
            resolution_scores[phrase] = False  # Default to unresolved

    return resolution_scores

# Function to check if an issue is ongoing
def detect_ongoing_issue(category_phrases):
    """Checks if an unresolved issue is ongoing."""
    ongoing_scores = {}

    for phrase in category_phrases:
        prompt = f"Does the customer describe an ongoing issue? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            ongoing_scores[phrase] = response.startswith("yes")  # True if ongoing, False otherwise
        except Exception:
            ongoing_scores[phrase] = False  # Default to past issue

    return ongoing_scores

# Function to check if the customer is demanding something
def detect_customer_demand(category_phrases):
    """Detects whether the customer is making a demand for resolution or compensation."""
    demand_scores = {}

    for phrase in category_phrases:
        prompt = f"Is the customer demanding compensation, a fix, or urgent action? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            demand_scores[phrase] = response.startswith("yes")  # True if demanding, False otherwise
        except Exception:
            demand_scores[phrase] = False  # Default to no demand

    return demand_scores

# ---- RUN THE PIPELINE ----
start_time = time.time()

# Extract unique phrases for efficiency
unique_category_phrases = list(set(" ".join(phrases) for phrases in complaint_analysis['category_phrases'] if phrases))

# Step 1: Run Issue Resolution Detection
print("🔍 Running Issue Resolution Detection...")
resolution_scores = detect_issue_resolution(unique_category_phrases)
print(f"✅ Completed Resolution Detection - {time.time() - start_time:.2f}s")

# Assign issue_resolved column
complaint_analysis['issue_resolved'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: resolution_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Filter unresolved issues for further analysis
unresolved_phrases = [phrase for phrase in unique_category_phrases if not resolution_scores.get(phrase, False)]

# Step 2: Run Ongoing Issue Detection on unresolved issues
print("🔍 Running Ongoing Issue Detection...")
ongoing_scores = detect_ongoing_issue(unresolved_phrases)
print(f"✅ Completed Ongoing Issue Detection - {time.time() - start_time:.2f}s")

# Assign ongoing_issue column
complaint_analysis['ongoing_issue'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: ongoing_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Step 3: Run Customer Demand Detection on unresolved issues
print("🔍 Running Customer Demand Detection...")
demand_scores = detect_customer_demand(unresolved_phrases)
print(f"✅ Completed Customer Demand Detection - {time.time() - start_time:.2f}s")

# Assign customer_demand column
complaint_analysis['customer_demand'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: demand_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Show Results
print(f"✅ Full Pipeline Completed - {time.time() - start_time:.2f}s")
complaint_analysis

Device set to use cpu


🔍 Running Issue Resolution Detection...
✅ Completed Resolution Detection - 28.96s
🔍 Running Ongoing Issue Detection...
✅ Completed Ongoing Issue Detection - 49.09s
🔍 Running Customer Demand Detection...
✅ Completed Customer Demand Detection - 69.66s
✅ Full Pipeline Completed - 69.69s


,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,data_usage_2021-01-01 00:00:00,data_usage_2021-02-01 00:00:00,data_usage_2021-03-01 00:00:00,data_usage_2021-04-01 00:00:00,data_usage_2021-05-01 00:00:00,data_usage_2021-06-01 00:00:00,data_usage_2021-07-01 00:00:00,data_usage_2021-08-01 00:00:00,data_usage_2021-09-01 00:00:00,data_usage_2021-10-01 00:00:00,data_usage_2021-11-01 00:00:00,data_usage_2021-12-01 00:00:00,phone_usage_2021-01-01 00:00:00,phone_usage_2021-02-01 00:00:00,phone_usage_2021-03-01 00:00:00,phone_usage_2021-04-01 00:00:00,phone_usage_2021-05-01 00:00:00,phone_usage_2021-06-01 00:00:00,phone_usage_2021-07-01 00:00:00,phone_usage_2021-08-01 00:00:00,phone_usage_2021-09-01 00:00:00,phone_usage_2021-10-01 00:00:00,phone_usage_2021-11-01 00:00:00,phone_usage_2021-12-01 00:00:00,use_app_2021-01-01 00:00:00,use_app_2021-02-01 00:00:00,use_app_2021-03-01 00:00:00,use_app_2021-04-01 00:00:00,use_app_2021-05-01 00:00:00,use_app_2021-06-01 00:00:00,use_app_2021-07-01 00:00:00,use_app_2021-08-01 00:00:00,use_app_2021-09-01 00:00:00,use_app_2021-10-01 00:00:00,use_app_2021-11-01 00:00:00,use_app_2021-12-01 00:00:00,complaint,category,complaint_boolean,monthly_avg_data,monthly_avg_phone,monthly_avg_app,app_usage_bins,phone_usage_bins,current_age,join_month,join_quarter,general_phrases,category_phrases,issue_resolved,severe_case,negativity_score,ongoing_issue,past_issue,customer_demand
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,2.07,13.87,12.11,3.40,12.22,21.06,4.29,10.17,21.43,8.85,16.34,21.63,2038.61,1952.95,1762.70,1383.54,217.01,174.23,1983.84,799.41,3371.37,2168.73,227.18,4920.68,0,1,1,1,1,1,1,1,1,1,1,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10001, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: There have been frequent interruptions to my service, and I am unable to rely on the connection..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. This situation has greatly impacted my ability to carry out work from home.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Service Disruption,1,12.286667,1750.020833,0.916667,very_high,lower 2200,26.975342,01_2015,Q1_2015,[This situation has greatly impacted my ability to carry out work from home],"[There have been frequent interruptions to my service, and I am unable to rely on the connection]",False,False,1,True,False,False
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,15.28,40.19,27.20,17.89,48.10,41.49,43.77,42.63,42.63,46.94,26.22,39.68,4627.57,1785.52,4617.92,2639.51,2159.29,3285.00,2919.12,4636.01,186.52,243.33,2102.91,236.99,1,1,0,1,1,1,1,1,0,1,0,1,"Subject: Official Complaint.\n\n The undersigned, customer with code 10004, is submitting an official complaint regarding the service. For the past few weeks, I have encountered the following issue: My internet connection was unstable last week, but after contacting technical support, the issue was resolved..\n \n Despite reaching out to customer support multiple times, the issue remains unresolved. The lack of resolution is causing frustration and affecting my usage of the service.\n \n I kindly request that you take immediate action to resolve this issue and provide a definitive solution.\n \n I look forward to your prompt response.",Connection Problem,1,36.001667,2453.307500,0.750000,medium,2450-2500,51.624658,05_2019,Q2_2019,[The lack of resolution is causing frustration and affecting my usage of the service],"[My internet connection was unstable last week, but after contacting technical support, the issue was resolved]",True,False,1,False,False,False
12,10012,2004-03-16,2018-02-04,1,prepaid,0,0,1,13,3.909589,9.29,16.31,12.54,46.52,18.15,19.19,4.86,30.72,2

In [570]:
sample_df

,category_phrases,issue_description,issue_resolved,past_issue_explanation
4823,"[A temporary service outage occurred, but the issue was handled professionally by the support team]",no,True,no
7098,"[The support team restored the service quickly after a brief outage, and I appreciate the swift response]",I appreciate the swift response,True,no
6465,"[Despite my repeated attempts to have these charges corrected, nothing has been done, and I am still being overcharged]",a refund,False,no
5655,"[I had an issue with my bill last month, but customer service quickly resolved the matter and refunded the charge]",a refund,True,yes
910,"[I was charged incorrectly, but customer support issued a refund, and the issue was resolved]",a refund,True,yes
...,...,...,...,...
7565,"[I had some trouble with my internet connection, but the support team fixed it quickly]",I will call the customer service number,True,no
843,"[I’ve been billed for services that I never used, and I have yet to receive an explanation or a refund]",a refund,False,no
3655,"[There was an unauthorized charge on my bill, but customer support refunded the amount quickly]",a refund,True,no
5097,"[I’ve experienced several service interruptions in the past few weeks, making it difficult to rely on your service]",I will call you to fix the issue,False,no


In [468]:
import time
from transformers import pipeline
import pandas as pd

# Load LLM pipeline
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

def detect_issue_resolution(category_phrases):
    """Classifies whether an issue was resolved."""
    resolution_scores = {}

    for phrase in category_phrases:
        prompt = f"Was the issue resolved? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            resolution_scores[phrase] = response.startswith("yes")  # True if "Yes", False otherwise
        except Exception:
            resolution_scores[phrase] = False  # Default to unresolved if error occurs

    return resolution_scores

def assess_negativity(category_phrases, general_phrases):
    """Detects whether phrases express negativity."""
    category_negative_scores = {}
    general_negative_scores = {}

    for phrase in category_phrases:
        prompt = f"Does this phrase express a negative sentiment? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            category_negative_scores[phrase] = response.startswith("yes")
        except Exception:
            category_negative_scores[phrase] = False  # Default to non-negative

    for phrase in general_phrases:
        prompt = f"Does this phrase express a negative sentiment? Only answer 'Yes' or 'No'. Context: {phrase}"
        try:
            response = llm_pipeline(prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            general_negative_scores[phrase] = response.startswith("yes")
        except Exception:
            general_negative_scores[phrase] = False  # Default to non-negative

    return category_negative_scores, general_negative_scores

def detect_customer_needs(category_phrases):
    """Extracts whether the customer needs tech or billing support, or dislikes the conditions."""
    dislike_conditions = {}

    for phrase in category_phrases:
        # Dislike Conditions
        conditions_prompt = f"Is the customer unhappy with the terms, conditions, or contract details? Only answer 'Yes' or 'No'. Context: {phrase}"

        try:
            conditions_response = llm_pipeline(conditions_prompt, max_length=5, do_sample=False)[0]['generated_text'].strip().lower()
            dislike_conditions[phrase] = conditions_response.startswith("yes")

        except Exception:
            dislike_conditions[phrase] = False  # Default fallback

    return dislike_conditions

# ---- RUN THE PIPELINE ----
start_time = time.time()

# Extract unique phrases for efficiency
unique_category_phrases = list(set(" ".join(phrases) for phrases in complaint_analysis['category_phrases'] if phrases))
unique_general_phrases = list(set(" ".join(phrases) for phrases in complaint_analysis['general_phrases'] if phrases))

# Run Resolution Detection
print("Running Issue Resolution Detection...")
resolution_scores = detect_issue_resolution(unique_category_phrases)
print(f"✅ Completed Resolution Detection - {time.time() - start_time:.2f}s")

# Assign issue_resolved column
complaint_analysis['issue_resolved'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: resolution_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Run Negativity Detection
print("Running Sentiment Assessment...")
category_negative_scores, general_negative_scores = assess_negativity(unique_category_phrases, unique_general_phrases)
print(f"✅ Completed Sentiment Assessment - {time.time() - start_time:.2f}s")

# Assign negativity columns
complaint_analysis['category_phrase_negative'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: category_negative_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

complaint_analysis['general_phrase_negative'] = complaint_analysis['general_phrases'].apply(
    lambda phrases: general_negative_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Run Customer Needs Detection
print("Running Customer Needs Detection...")
dislike_conditions_scores = detect_customer_needs(unique_category_phrases)
print(f"✅ Completed Customer Needs Assessment - {time.time() - start_time:.2f}s")

# Assign customer needs columns

complaint_analysis['dislike_conditions'] = complaint_analysis['category_phrases'].apply(
    lambda phrases: dislike_conditions_scores.get(" ".join(phrases), False) if isinstance(phrases, list) else False
)

# Show Results

print(f"✅ Full Pipeline Completed - {time.time() - start_time:.2f}s")
complaint_analysis

Device set to use cpu


Running Issue Resolution Detection...
✅ Completed Resolution Detection - 31.58s
Running Sentiment Assessment...
✅ Completed Sentiment Assessment - 67.17s
Running Customer Needs Detection...
✅ Completed Customer Needs Assessment - 97.48s
✅ Full Pipeline Completed - 97.50s


,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,phone_usage_bins,current_age,join_month,join_quarter,general_phrases,category_phrases,issue_resolved,category_phrase_negative,general_phrase_negative,dislike_conditions
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,...,lower 2200,26.975342,01_2015,Q1_2015,[This situation has greatly impacted my abilit...,[There have been frequent interruptions to my ...,False,True,True,True
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,...,2450-2500,51.624658,05_2019,Q2_2019,[The lack of resolution is causing frustration...,[My internet connection was unstable last week...,True,False,True,False
12,10012,2004-03-16,2018-02-04,1,prepaid,0,0,1,13,3.909589,...,lower 2200,16.909589,02_2018,Q1_2018,[This situation has greatly impacted my abilit...,[There are unauthorized charges on my account ...,False,True,True,True
18,10018,1999-10-19,2015-06-05,0,postpaid,0,1,0,15,6.580822,...,2400-2450,21.580822,06_2015,Q2_2015,"[As a result of this issue, I have missed impo...",[I had some trouble with my internet connectio...,True,False,True,False
19,10019,1971-02-22,2012-06-26,1,prepaid,0,0,1,41,9.523288,...,2200-2250,50.523288,06_2012,Q2_2012,[This ongoing problem has caused significant d...,[I feel misled by the promotional offers made ...,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9979,19979,1987-12-23,2012-01-16,0,postpaid,0,1,0,24,9.967123,...,over 2500,33.967123,01_2012,Q1_2012,[This ongoing problem has caused significant d...,"[A temporary service outage occurred, but the ...",True,False,True,False
9980,19980,1972-06-08,2012-01-24,0,prepaid,0,0,1,39,9.945205,...,over 2500,48.945205,01_2012,Q1_2012,"[but it was resolved within a few hours, This ...",[I experienced a brief disruption to my phone ...,False,False,False,False
9986,19986,1985-04-23,2012-06-04,1,postpaid,0,1,0,27,9.583562,...,2250-2300,36.583562,06_2012,Q2_2012,[The lack of resolution is causing frustration...,"[Over the past few months, I have been overcha...",False,True,True,True
9990,19990,1992-11-29,2012-07-23,1,pay-as-you-go,1,0,0,19,9.449315,...,lower 2200,28.449315,07_2012,Q3_2012,[This ongoing problem has caused significant d...,[I signed up because of the promotional discou...,False,True,True,True


In [324]:
import pandas as pd

# Step 1️⃣: Create a feature matrix from the complaint_analysis
feature_columns = [
    "category_phrase_negative",
    "general_phrase_negative",
    "dislike_conditions"
]

# Step 2️⃣: Count unresolved issues per customer
unresolved_issues_count = (
    complaint_analysis[~complaint_analysis["issue_resolved"]]
    .groupby("customer_id")
    .size()
    .reset_index(name="unresolved_issue_count")
)

# Ensure "unresolved_issue_count" does not exist in merged_data before merging
if "unresolved_issue_count" in merged_data.columns:
    merged_data.drop(columns=["unresolved_issue_count"], inplace=True)

# Step 3️⃣: Collect feature matrices for unresolved complaints per customer
unresolved_complaints = (
    complaint_analysis[~complaint_analysis["issue_resolved"]]
    .groupby("customer_id")
    .apply(lambda df: df[feature_columns].to_dict(orient="records"))
    .reset_index(name="complaint_features")
)

# Ensure "complaint_features" does not exist in merged_data before merging
if "complaint_features" in merged_data.columns:
    merged_data.drop(columns=["complaint_features"], inplace=True)

# Step 4️⃣: Merge the aggregated data into merged_data
merged_data = merged_data.merge(unresolved_issues_count, on="customer_id", how="left").fillna({"unresolved_issue_count": 0})
merged_data = merged_data.merge(unresolved_complaints, on="customer_id", how="left")

# Step 5️⃣: Ensure "complaint_features" is always a list
merged_data["complaint_features"] = merged_data["complaint_features"].apply(lambda x: x if isinstance(x, list) else [])

# ✅ Display final structured dataset
merged_data

C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\1970506457.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df[feature_columns].to_dict(orient="records"))


,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,monthly_avg_data,monthly_avg_phone,monthly_avg_app,app_usage_bins,phone_usage_bins,current_age,join_month,join_quarter,unresolved_issue_count,complaint_features
0,10000,1994-08-13,2015-11-22,0,postpaid,0,1,0,21,6.115068,...,76.915833,3017.853333,0.666667,medium,over 2500,27.115068,11_2015,Q4_2015,0.0,[]
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,...,12.286667,1750.020833,0.916667,very_high,lower 2200,26.975342,01_2015,Q1_2015,1.0,"[{'category_phrase_negative': True, 'general_p..."
2,10002,2008-06-10,2020-05-22,0,prepaid,0,0,1,11,1.613699,...,36.716667,2856.807500,0.583333,low,over 2500,12.613699,05_2020,Q2_2020,0.0,[]
3,10003,1970-09-04,2017-11-10,0,prepaid,0,0,1,47,4.145205,...,31.731667,2584.227500,0.916667,very_high,over 2500,51.145205,11_2017,Q4_2017,0.0,[]
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,...,36.001667,2453.307500,0.750000,medium,2450-2500,51.624658,05_2019,Q2_2019,0.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19995,2007-07-30,2015-11-23,1,prepaid,0,0,1,8,6.112329,...,31.421667,2249.589167,0.666667,medium,2200-2250,14.112329,11_2015,Q4_2015,1.0,"[{'category_phrase_negative': True, 'general_p..."
9996,19996,1981-10-26,2018-03-18,0,prepaid,0,0,1,36,3.794521,...,36.510833,2723.397500,0.916667,very_high,over 2500,39.794521,03_2018,Q1_2018,0.0,[]
9997,19997,1999-01-10,2012-12-02,0,postpaid,0,1,0,13,9.087671,...,78.594167,2496.624167,0.666667,medium,2450-2500,22.087671,12_2012,Q4_2012,0.0,[]
9998,19998,1993-09-10,2015-09-09,0,postpaid,0,1,0,22,6.317808,...,52.725000,3131.480833,0.750000,medium,over 2500,28.317808,09_2015,Q3_2015,0.0,[]


### New features (replace old bins)

In [325]:
# Identify the last 4 months dynamically
app_usage_cols = [col for col in merged_data.columns if col.startswith("use_app_")]

last_4_months = sorted(app_usage_cols, key=lambda x: x[-10:])[-4:]

# Count how many of the last 4 months each user was active
merged_data["active_last_4mos"] = merged_data[last_4_months].sum(axis=1)

# Define active users as those with more than 2 months of usage in the last 4 months
merged_data["recent_active_status"] = (merged_data["active_last_4mos"] > 2).astype(int)

In [326]:
import numpy as np

# Calculate the 70th percentile for data usage (top 30% threshold)
data_usage_threshold = np.percentile(merged_data["monthly_avg_data"].dropna(), 70)

# Calculate the 30th percentile for phone usage (bottom 30% threshold)
phone_usage_threshold = np.percentile(merged_data["monthly_avg_phone"].dropna(), 30)

# Create the features
merged_data["data_usage_high"] = merged_data["monthly_avg_data"] >= data_usage_threshold
merged_data["phone_usage_low"] = merged_data["monthly_avg_phone"] <= phone_usage_threshold

# Display the updated dataset
merged_data

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,phone_usage_bins,current_age,join_month,join_quarter,unresolved_issue_count,complaint_features,active_last_4mos,recent_active_status,data_usage_high,phone_usage_low
0,10000,1994-08-13,2015-11-22,0,postpaid,0,1,0,21,6.115068,...,over 2500,27.115068,11_2015,Q4_2015,0.0,[],3,1,True,False
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,...,lower 2200,26.975342,01_2015,Q1_2015,1.0,"[{'category_phrase_negative': True, 'general_p...",4,1,False,True
2,10002,2008-06-10,2020-05-22,0,prepaid,0,0,1,11,1.613699,...,over 2500,12.613699,05_2020,Q2_2020,0.0,[],4,1,True,False
3,10003,1970-09-04,2017-11-10,0,prepaid,0,0,1,47,4.145205,...,over 2500,51.145205,11_2017,Q4_2017,0.0,[],4,1,False,False
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,...,2450-2500,51.624658,05_2019,Q2_2019,0.0,[],2,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19995,2007-07-30,2015-11-23,1,prepaid,0,0,1,8,6.112329,...,2200-2250,14.112329,11_2015,Q4_2015,1.0,"[{'category_phrase_negative': True, 'general_p...",3,1,False,True
9996,19996,1981-10-26,2018-03-18,0,prepaid,0,0,1,36,3.794521,...,over 2500,39.794521,03_2018,Q1_2018,0.0,[],3,1,True,False
9997,19997,1999-01-10,2012-12-02,0,postpaid,0,1,0,13,9.087671,...,2450-2500,22.087671,12_2012,Q4_2012,0.0,[],4,1,True,False
9998,19998,1993-09-10,2015-09-09,0,postpaid,0,1,0,22,6.317808,...,over 2500,28.317808,09_2015,Q3_2015,0.0,[],2,0,True,False


In [327]:
import pandas as pd

# Assuming merged_data is a pandas DataFrame
summary = merged_data.groupby('plan_type')['monthly_avg_data'].agg(['mean', 'min', 'max'])

# Display the summary
summary

,mean,min,max
plan_type,,,
pay-as-you-go,15.611766,7.295000,24.353333
postpaid,62.738026,31.100833,91.204167
prepaid,31.228732,13.829167,48.869167


In [328]:
# Load the dataset (assuming it's already loaded as 'merged_data')

# Assign churn probability based on 'churn_in_3_mos' feature
merged_data["churn_proba"] = merged_data["churn_in_3mos"].apply(lambda x: 0.8 if x == 1 else 0.2)

merged_data

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,current_age,join_month,join_quarter,unresolved_issue_count,complaint_features,active_last_4mos,recent_active_status,data_usage_high,phone_usage_low,churn_proba
0,10000,1994-08-13,2015-11-22,0,postpaid,0,1,0,21,6.115068,...,27.115068,11_2015,Q4_2015,0.0,[],3,1,True,False,0.2
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,...,26.975342,01_2015,Q1_2015,1.0,"[{'category_phrase_negative': True, 'general_p...",4,1,False,True,0.8
2,10002,2008-06-10,2020-05-22,0,prepaid,0,0,1,11,1.613699,...,12.613699,05_2020,Q2_2020,0.0,[],4,1,True,False,0.2
3,10003,1970-09-04,2017-11-10,0,prepaid,0,0,1,47,4.145205,...,51.145205,11_2017,Q4_2017,0.0,[],4,1,False,False,0.2
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,...,51.624658,05_2019,Q2_2019,0.0,[],2,0,False,False,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19995,2007-07-30,2015-11-23,1,prepaid,0,0,1,8,6.112329,...,14.112329,11_2015,Q4_2015,1.0,"[{'category_phrase_negative': True, 'general_p...",3,1,False,True,0.8
9996,19996,1981-10-26,2018-03-18,0,prepaid,0,0,1,36,3.794521,...,39.794521,03_2018,Q1_2018,0.0,[],3,1,True,False,0.2
9997,19997,1999-01-10,2012-12-02,0,postpaid,0,1,0,13,9.087671,...,22.087671,12_2012,Q4_2012,0.0,[],4,1,True,False,0.2
9998,19998,1993-09-10,2015-09-09,0,postpaid,0,1,0,22,6.317808,...,28.317808,09_2015,Q3_2015,0.0,[],2,0,True,False,0.2


In [329]:
# Ensure that `complaint_analysis` and `merged_data` are properly loaded

# Get the unique complaint categories
complaint_categories = complaint_analysis["category"].unique()

# Initialize new boolean features for each complaint category in merged_data
for category in complaint_categories:
    merged_data[f"complaint_{category}"] = False

# Iterate over unresolved complaints and mark them in merged_data
for _, complaint in complaint_analysis.iterrows():
    if not complaint["issue_resolved"]:  # Check if the issue is unresolved
        customer_id = complaint["customer_id"]
        category = complaint["category"]
        
        # Update the corresponding boolean feature in merged_data
        merged_data.loc[merged_data["customer_id"] == customer_id, f"complaint_{category}"] = True

merged_data

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,active_last_4mos,recent_active_status,data_usage_high,phone_usage_low,churn_proba,complaint_Service Disruption,complaint_Connection Problem,complaint_Unauthorized Charges,complaint_Promotional Issue,complaint_Billing Issue
0,10000,1994-08-13,2015-11-22,0,postpaid,0,1,0,21,6.115068,...,3,1,True,False,0.2,False,False,False,False,False
1,10001,1994-06-25,2015-01-12,1,pay-as-you-go,1,0,0,20,6.975342,...,4,1,False,True,0.8,True,False,False,False,False
2,10002,2008-06-10,2020-05-22,0,prepaid,0,0,1,11,1.613699,...,4,1,True,False,0.2,False,False,False,False,False
3,10003,1970-09-04,2017-11-10,0,prepaid,0,0,1,47,4.145205,...,4,1,False,False,0.2,False,False,False,False,False
4,10004,1969-11-06,2019-05-19,0,prepaid,0,0,1,49,2.624658,...,2,0,False,False,0.2,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19995,2007-07-30,2015-11-23,1,prepaid,0,0,1,8,6.112329,...,3,1,False,True,0.8,False,False,False,True,False
9996,19996,1981-10-26,2018-03-18,0,prepaid,0,0,1,36,3.794521,...,3,1,True,False,0.2,False,False,False,False,False
9997,19997,1999-01-10,2012-12-02,0,postpaid,0,1,0,13,9.087671,...,4,1,True,False,0.2,False,False,False,False,False
9998,19998,1993-09-10,2015-09-09,0,postpaid,0,1,0,22,6.317808,...,2,0,True,False,0.2,False,False,False,False,False


Response LLM

In [330]:
# Extract a subset of 100 customers with churn probability above 0.75 for testing
response_testing_df = merged_data[merged_data["churn_proba"] > 0.75].sample(n=100)

response_testing_df

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,active_last_4mos,recent_active_status,data_usage_high,phone_usage_low,churn_proba,complaint_Service Disruption,complaint_Connection Problem,complaint_Unauthorized Charges,complaint_Promotional Issue,complaint_Billing Issue
9759,19759,1991-05-19,2016-07-05,1,postpaid,0,1,0,25,5.495890,...,1,0,True,True,0.8,False,False,True,False,False
8942,18942,1976-11-11,2014-08-23,1,prepaid,0,0,1,37,7.364384,...,4,1,False,True,0.8,False,False,False,False,False
7772,17772,2003-03-05,2019-09-19,1,prepaid,0,0,1,16,2.287671,...,4,1,False,True,0.8,False,True,False,False,False
1366,11366,1961-03-19,2018-08-18,1,prepaid,0,0,1,57,3.375342,...,4,1,False,True,0.8,False,False,False,True,False
6417,16417,1979-11-30,2019-03-28,1,prepaid,0,0,1,39,2.767123,...,2,0,False,True,0.8,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9299,19299,1969-10-29,2020-03-10,1,prepaid,0,0,1,50,1.813699,...,4,1,False,True,0.8,False,False,False,False,False
6651,16651,1985-04-11,2014-09-13,1,prepaid,0,0,1,29,7.306849,...,4,1,False,True,0.8,False,False,False,False,True
1652,11652,1974-01-08,2014-05-24,1,prepaid,0,0,1,40,7.613699,...,4,1,False,True,0.8,False,False,False,True,False
5853,15853,2000-03-21,2013-06-12,1,prepaid,0,0,1,13,8.561644,...,3,1,False,True,0.8,True,False,False,False,False


In [331]:
response_testing_df["plan_type"]

9759    postpaid
8942     prepaid
7772     prepaid
1366     prepaid
6417     prepaid
          ...   
9299     prepaid
6651     prepaid
1652     prepaid
5853     prepaid
972      prepaid
Name: plan_type, Length: 100, dtype: object

In [169]:
import pandas as pd

# Load the Excel file into a DataFrame
file_path = "Retention Decision Rules.xlsx"  # Update with the correct path
decisions_df = pd.read_excel(file_path, sheet_name="Decision Rules Retention")

# Display the first few rows to check if it loaded correctly
decisions_df.head()

,plan_type,test_group,complaint_Billing Issue,complaint_Connection Problem,complaint_Service Disruption,complaint_Unauthorized Charges,complaint_Promotional Issue,case_type,data_usage_high,recent_active_status,...,X-Sell Candidate,activation_message,forward_department,channel_immediate_action,content_immediate_action,follow_up_time,follow_up_channel,follow_up_content,churn_stay_offer_channel,churn_stay_offer_content
0,postpaid,A,True,False,False,False,False,Support Case,True,1,...,True,False,Billing Department,Push + In-App Banner,Direct issue resolution + 1 month Partner vouc...,t + 1 month,Push + In-App Banner,Offer to keep eSIM with discount in first 3 mo...,Any chosen platform,"1 month free eSim (for gadget, tablet or travel)"
1,postpaid,B,True,False,False,False,False,Support Case,True,1,...,True,False,Billing Department,Push + In-App Banner,Direct issue resolution + 1 month free eSim (f...,t + 1 month,Push + In-App Banner,Stay-offer for partner content at discount,Any chosen platform,"1 month Partner voucher (Streaming, Music, etc.)"
2,postpaid,A,False,True,False,False,False,Support Case,True,1,...,True,False,Tech Support,Push + In-App Banner,Direct issue resolution + 1 month Partner vouc...,t + 1 month,Push + In-App Banner,Offer to keep eSIM with discount in first 3 mo...,Any chosen platform,"1 month free eSim (for gadget, tablet or travel)"
3,postpaid,B,False,True,False,False,False,Support Case,True,1,...,True,False,Tech Support,Push + In-App Banner,Direct issue resolution + 1 month free eSim (f...,t + 1 month,Push + In-App Banner,Stay-offer for partner content at discount,Any chosen platform,"1 month Partner voucher (Streaming, Music, etc.)"
4,postpaid,A,False,False,False,False,True,Retention Case,True,1,...,True,False,CRM,Push + In-App Banner,"1 month Partner voucher (Streaming, Music, etc.)",t + 1 month,Push + In-App Banner,Stay-offer for partner content at discount,Any chosen platform,"1 month free eSim (for gadget, tablet or travel)"


In [380]:
import json

def convert_to_json_with_forward_department(df):
    """Final JSON with forward_department in Immediate Actions."""
    decision_tree = {}

    for _, row in df.iterrows():
        # Determine Support Category
        support_categories = []
        if row["complaint_Service Disruption"] or row["complaint_Connection Problem"]:
            support_categories.append("Technical Support")
        if row["complaint_Unauthorized Charges"] or row["complaint_Billing Issue"]:
            support_categories.append("Administrative Support")
        if row["complaint_Promotional Issue"]:
            support_categories.append("Promotional Issue")
        if not support_categories:
            support_categories.append("No Support Issue")  # If no complaints exist

        # Use the first matching support category
        support_category = support_categories[0]

        # Extract key decision features
        plan = row["plan_type"]
        data_usage = "High" if row["data_usage_high"] else "Low"
        active_status = "Active" if row["recent_active_status"] == 1 else "Inactive"
        test_group = row["test_group"]

        # Extract upsell/x-sell goal
        if row["upsell"]:
            goal = "Upsell"
        elif row["X-Sell Candidate"]:
            goal = "X-Sell"
        else:
            goal = "Retention"

        # Ensure structure exists
        decision_tree.setdefault(support_category, {}) \
            .setdefault(plan, {}) \
            .setdefault(data_usage, {}) \
            .setdefault(active_status, {}) \
            .setdefault(test_group, {})

        # Store retention actionsa
        decision_tree[support_category][plan][data_usage][active_status][test_group] = {
            "Immediate Action": {
                "Content": row["content_immediate_action"],
                "Channel": row["channel_immediate_action"],
                "Involved Department": row["forward_department"],
                "Goal": "Retention"
            },
            "Follow-Up": {
                "Time": row["follow_up_time"],
                "Channel": row["follow_up_channel"],
                "Content": row["follow_up_content"],
                "Goal": goal  
            },
            "Churn Stay-Offer": {
                "Channel": row["churn_stay_offer_channel"],
                "Content": row["churn_stay_offer_content"],
                "Goal": "Retention"
            }
        }

    return decision_tree

# Generate Final JSON with Forward Department
decision_rules_json_final = convert_to_json_with_forward_department(decisions_df)

# Save JSON file
json_file_path = "Retention_Decision_Rules_With_Forward_Department.json"
with open(json_file_path, "w") as json_file:
    json.dump(decision_rules_json_final, json_file, indent=4)

# Print a sample of the final JSON for verification
print(json.dumps(decision_rules_json_final, indent=4))

{
    "Administrative Support": {
        "postpaid": {
            "High": {
                "Active": {
                    "A": {
                        "Immediate Action": {
                            "Content": "Direct issue resolution + 1 month Partner voucher (Streaming, Music, etc.)",
                            "Channel": "Push + In-App Banner",
                            "Involved Department": "Billing Department",
                            "Goal": "Retention"
                        },
                        "Follow-Up": {
                            "Time": "t + 1 month",
                            "Channel": "Push + In-App Banner",
                            "Content": "Offer to keep eSIM with discount in first 3 months",
                            "Goal": "X-Sell"
                        },
                        "Churn Stay-Offer": {
                            "Channel": "Any chosen platform",
                            "Content": "1 month free eSim (for gadg

In [381]:
import random

def extract_messages_with_metadata(customer_row, decision_rules_json):
    """
    Extracts the content, channel, department, time, and goal for Immediate Action, Follow-Up,
    and Churn Stay-Offer based on customer data and random A/B assignment.
    """
    # Get the mapped profile (which contains the customer characteristics)
    mapped_profile = customer_row["mapped_profile"]
    
    # Randomly assign test group: A or B
    test_group = random.choice(["A", "B"])
    
    # Extract customer features
    plan_type = customer_row["plan_type"]
    data_usage = "High" if customer_row["data_usage_high"] else "Low"
    active_status = "Active" if customer_row["recent_active_status"] == 1 else "Inactive"
    support_category = mapped_profile["support_category"]
    
    # Look up the decision rule based on the customer profile and test group
    action_data = decision_rules_json.get(plan_type, {}).get(test_group, {}).get(support_category, {}).get(data_usage, {}).get(active_status, {})
    
    # Extract content for Immediate Action, Follow-Up, and Churn Stay-Offer
    immediate_action = action_data.get("Immediate Action", {})
    follow_up = action_data.get("Follow-Up", {})
    churn_stay_offer = action_data.get("Churn Stay-Offer", {})
    
    # Prepare the extracted data in the required format
    immediate_action_content = immediate_action.get("Content", "No Immediate Action")
    immediate_action_channel = immediate_action.get("Channel", "No Channel")
    immediate_action_department = immediate_action.get("Involved Department", "No Department")
    immediate_action_goal = immediate_action.get("Goal", "No Goal")
    
    follow_up_time = follow_up.get("Time", "No Time")
    follow_up_channel = follow_up.get("Channel", "No Channel")
    follow_up_content = follow_up.get("Content", "No Content")
    follow_up_goal = follow_up.get("Goal", "No Goal")
    
    churn_stay_offer_channel = churn_stay_offer.get("Channel", "No Channel")
    churn_stay_offer_content = churn_stay_offer.get("Content", "No Content")
    churn_stay_offer_goal = churn_stay_offer.get("Goal", "No Goal")
    
    # Return the relevant message content and metadata for each action type in a readable format
    return {
        "immediate_action": f"Content: {immediate_action_content}, Channel: {immediate_action_channel}, Involved Department: {immediate_action_department}, Goal: {immediate_action_goal}",
        "follow_up": f"Time: {follow_up_time}, Channel: {follow_up_channel}, Content: {follow_up_content}, Goal: {follow_up_goal}",
        "churn_stay_offer": f"Channel: {churn_stay_offer_channel}, Content: {churn_stay_offer_content}, Goal: {churn_stay_offer_goal}"
    }

# Apply the function to all customers and populate the respective columns
def assign_messages_with_metadata_to_customers(df, decision_rules_json):
    # Extract and store the results in the corresponding columns
    df[['immediate_action', 'follow_up', 'churn_stay_offer']] = df.apply(
        lambda row: pd.Series(extract_messages_with_metadata(row, decision_rules_json)), axis=1
    )
    
    return df

# Assuming decision_rules_json is already available
response_testing_df = assign_messages_with_metadata_to_customers(response_testing_df, decision_rules_json)

# Display the updated dataframe with the new message content and metadata
print(response_testing_df[['customer_id', 'immediate_action', 'follow_up', 'churn_stay_offer']].head())

      customer_id                                   immediate_action  \
9759        19759  Content: Direct issue resolution + 1 month Par...   
8942        18942  Content: Activation message 50 International m...   
7772        17772  Content: Direct issue resolution + 1 month fre...   
1366        11366  Content: 5GB extra for 1 month, Channel: Push ...   
6417        16417  Content: Direct issue resolution + 5GB extra f...   

                                              follow_up  \
9759  Time: t + 1 month, Channel: Email + SMS, Conte...   
8942  Time: t + 1 month, Channel: Push + In-App Bann...   
7772  Time: t + 1 month, Channel: Push + In-App Bann...   
1366  Time: t + 1 month, Channel: Push + In-App Bann...   
6417  Time: t + 1 month, Channel: Email + SMS, Conte...   

                                       churn_stay_offer  
9759  Channel: Any chosen platform, Content: 1 month...  
8942  Channel: Any chosen platform, Content: 5GB ext...  
7772  Channel: Any chosen platform, Co

In [382]:
response_testing_df.head()

,customer_id,birth_date,join_date,churn_in_3mos,plan_type,plan_pay-as-you-go,plan_postpaid,plan_prepaid,age_at_joining,years_on_plan,...,complaint_Unauthorized Charges,complaint_Promotional Issue,complaint_Billing Issue,mapped_profile,retention_rule,llm_generated_messages,immediate_action,follow_up,churn_stay_offer,generated_messages
9759,19759,1991-05-19,2016-07-05,1,postpaid,0,1,0,25,5.495890,...,True,False,False,"{'support_category': 'Administrative Support',...",{'Immediate Action': {'Content': 'Direct issue...,📢 **Retention Strategy**\n \n ✅ ...,Content: Direct issue resolution + 1 month Par...,"Time: t + 1 month, Channel: Email + SMS, Conte...","Channel: Any chosen platform, Content: 1 month...","{'customer_id': 19759, 'test_group': 'B', 'imm..."
8942,18942,1976-11-11,2014-08-23,1,prepaid,0,0,1,37,7.364384,...,False,False,False,"{'support_category': 'No Support Issue', 'plan...",{'Immediate Action': {'Content': 'Activation m...,📢 **Retention Strategy**\n \n ✅ ...,Content: Activation message 50 International m...,"Time: t + 1 month, Channel: Push + In-App Bann...","Channel: Any chosen platform, Content: 5GB ext...","{'customer_id': 18942, 'test_group': 'B', 'imm..."
7772,17772,2003-03-05,2019-09-19,1,prepaid,0,0,1,16,2.287671,...,False,False,False,"{'support_category': 'Technical Support', 'pla...",{'Immediate Action': {'Content': 'Direct issue...,📢 **Retention Strategy**\n \n ✅ ...,Content: Direct issue resolution + 1 month fre...,"Time: t + 1 month, Channel: Push + In-App Bann...","Channel: Any chosen platform, Content: 5GB ext...","{'customer_id': 17772, 'test_group': 'A', 'imm..."
1366,11366,1961-03-19,2018-08-18,1,prepaid,0,0,1,57,3.375342,...,False,True,False,"{'support_category': 'Promotional Issue', 'pla...",{'Immediate Action': {'Content': '1 month free...,📢 **Retention Strategy**\n \n ✅ ...,"Content: 5GB extra for 1 month, Channel: Push ...","Time: t + 1 month, Channel: Push + In-App Bann...","Channel: Any chosen platform, Content: 1 month...","{'customer_id': 11366, 'test_group': 'B', 'imm..."
6417,16417,1979-11-30,2019-03-28,1,prepaid,0,0,1,39,2.767123,...,False,False,False,"{'support_category': 'Technical Support', 'pla...",{'Immediate Action': {'Content': 'Direct issue...,📢 **Retention Strategy**\n \n ✅ ...,Content: Direct issue resolution + 5GB extra f...,"Time: t + 1 month, Channel: Email + SMS, Conte...","Channel: Any chosen platform, Content: 1 month...","{'customer_id': 16417, 'test_group': 'B', 'imm..."


In [386]:
from transformers import pipeline
import pandas as pd
import random

# Load a pre-trained Hugging Face model (using a smaller and lighter model like 't5-small')
generator = pipeline("text-generation", model="t5-small")

def generate_message_text(value_type, unique_values):
    """Generates message text using Hugging Face's lightweight LLM for each unique value."""
    messages = []

    for value in unique_values:
        # Create a more detailed prompt to guide the model
        prompt = (
            f"Imagine you are a customer retention specialist at a telecommunications company. "
            f"A model has provided you with offer proposals to potential churners based on the following customer data. "
            f"Please create a personalized message for the customer based on the provided profile. "
            f"Your message should explain the offer, why it is relevant, and how it benefits the customer.\n\n"
            f"Customer Profile:\n{value}\n\n"
            f"Message Type: {value_type}\n\n"
            f"Instructions: Generate a message that fits the {value_type} category. "
            f"Make sure the tone is professional, engaging, and encourages the customer to take action. "
            f"Also, adapt the message to the communication channel specified. Use appropriate wording for SMS, email, or push notifications."
        )

        # Use Hugging Face's LLM to generate the message text
        response = generator(prompt, max_length=150, num_return_sequences=1)

        # Extract and store the generated message
        generated_message = response[0]["generated_text"]
        messages.append(generated_message)

    return messages


def generate_messages_for_columns(df):
    """Generates messages for Immediate Action, Follow-Up, and Churn Stay-Offer columns."""
    # Extract unique values for each message type
    immediate_action_values = df['immediate_action'].unique()
    follow_up_values = df['follow_up'].unique()
    churn_stay_offer_values = df['churn_stay_offer'].unique()

    # Generate messages for each of the columns
    immediate_action_messages = generate_message_text("Immediate Action", immediate_action_values)
    follow_up_messages = generate_message_text("Follow-Up", follow_up_values)
    churn_stay_offer_messages = generate_message_text("Churn Stay-Offer", churn_stay_offer_values)

    # Create a dictionary for easy mapping back to the columns
    messages_dict = {
        'immediate_action': dict(zip(immediate_action_values, immediate_action_messages)),
        'follow_up': dict(zip(follow_up_values, follow_up_messages)),
        'churn_stay_offer': dict(zip(churn_stay_offer_values, churn_stay_offer_messages))
    }

    return messages_dict


def assign_generated_messages(df, messages_dict):
    """Assign generated messages back to the original dataframe based on unique values."""
    # Loop through the dataframe rows and assign the appropriate message for each column
    for idx, row in df.iterrows():
        immediate_action_value = row['immediate_action']
        follow_up_value = row['follow_up']
        churn_stay_offer_value = row['churn_stay_offer']
        
        # Map the messages back to the dataframe
        df.at[idx, 'immediate_action_message'] = messages_dict['immediate_action'].get(immediate_action_value, '')
        df.at[idx, 'follow_up_message'] = messages_dict['follow_up'].get(follow_up_value, '')
        df.at[idx, 'churn_stay_offer_message'] = messages_dict['churn_stay_offer'].get(churn_stay_offer_value, '')
    
    return df


# Example usage on first 5 rows
sample_df = response_testing_df.head(5)  # You can change this to test a larger subset if needed
messages_dict = generate_messages_for_columns(sample_df)
sample_df = assign_generated_messages(sample_df, messages_dict)

# Display the updated dataframe with generated messages
print(sample_df[['customer_id', 'immediate_action_message', 'follow_up_message', 'churn_stay_offer_message']].head())

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCa

      customer_id                           immediate_action_message  \
9759        19759  Imagine you are a customer retention specialis...   
8942        18942  Imagine you are a customer retention specialis...   
7772        17772  Imagine you are a customer retention specialis...   
1366        11366  Imagine you are a customer retention specialis...   
6417        16417  Imagine you are a customer retention specialis...   

                                      follow_up_message  \
9759  Imagine you are a customer retention specialis...   
8942  Imagine you are a customer retention specialis...   
7772  Imagine you are a customer retention specialis...   
1366  Imagine you are a customer retention specialis...   
6417  Imagine you are a customer retention specialis...   

                               churn_stay_offer_message  
9759  Imagine you are a customer retention specialis...  
8942  Imagine you are a customer retention specialis...  
7772  Imagine you are a customer reten

C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\125335315.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[idx, 'immediate_action_message'] = messages_dict['immediate_action'].get(immediate_action_value, '')
C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\125335315.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[idx, 'follow_up_message'] = messages_dict['follow_up'].get(follow_up_value, '')
C:\Users\max-u.MTU-SFC\AppData\Local\Temp\ipykernel_3164\125335315.py:69: Setti